Author: Kevin ALBERT  
Created: June 2022  
Friends: [Marco Martins](https://github.com/marco-martins)  
QnA: [Brian W Bush](https://github.com/bwbush)

In [2]:
import datetime, time
print ('Last testrun on: ' + datetime.datetime.now().strftime("%d %b %Y (%H:%M)"))

Last testrun on: 10 Jan 2023 (16:58)


# Marlowe contract 
_**how to deploy a custom contract**_

## Contents
1. [Objective](#Objective)
1. [Installation](#Installation)
1. [Check](#Check)
1. [Wallets](#Wallets)
1. [Contract](#Contract)
1. [Funding](#Funding)
1. [Tokens](#Tokens)
1. [Deployment](#Deployment)


## Objective

Reference guide that explain and demonstrate what I learned from the <div class="ttooltip">Marlowe<span class="tooltiptext">a domain-specific language (DSL) built on Plutus that is used to write smart contracts for financial applications. It aims to make it easy for non-technical users to write smart contracts for financial applications such as crowdfunding, escrow, and lending.</span></div> Pioneer Program (summer 2022)  
on how to run a custom smart contract on the blockchain using marlowe-cli.  
Python and other modules will be used together with these cardano script command tools.  

### Tools

This notebook will use all these commands:

```text
  cardano-node run                               Run the node
  
  cardano-wallet serve                           Serve API that listens for commands/actions
  cardano-wallet recovery-phrase generate        Generate an English recovery phrase
  cardano-wallet key from-recovery-phrase        Convert a recovery phrase to an extended private key
  
  cardano-cli key convert-cardano-address-key    Convert a cardano-address extended signing key to a Shelley-format key
  cardano-cli key verification-key               Get a verification key from a signing key, supports all key types
  cardano-cli address build                      Build a Shelley payment address, optional delegation to a stake address
  cardano-cli query tip                          Get the node's current tip (slot no, hash, block no)
  cardano-cli query utxo                         Get a portion of the current UTxO: by tx in, by address or the whole
  
  marlowe-cli template swap                      Create a swap contract
  marlowe-cli util slotting                      Find the slot-to-time relationship for the current epoch
  marlowe-cli util mint                          Mint native tokens
  marlowe-cli util clean                         Reorganize the UTxOs at an address, separating tokens
  marlowe-cli transaction simple                 Build a non-Marlowe transaction
  marlowe-cli run initialize                     Init first transaction of Marlowe contract and write a JSON file
  marlowe-cli run execute                        Run a Marlowe transaction
  marlowe-cli run prepare                        Prep next step of a Marlowe contract and write a JSON file
  marlowe-cli run withdraw                       Withdraw funds from the Marlowe role address
```

### Network
The marlowe-pioneer blockchain testnet is offline since December 2022 (NetworkMagic: 1567).  
We therefore need to use [other Cardano blockchain networks](https://developers.cardano.org/docs/get-started/running-cardano/#cardano-blockchain--nets).  
  * testnet
    * preview (NetworkMagic: 2)
    * preprod (NetworkMagic: 1)
  * mainnet
    * mainnet (NetworkMagic: 764824073)
  
This notebook will use the **preview** Cardano blockchain network, using MAGIC **2**.  

### Contract
known contract templates:  
  * **[actus](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli/examples/actus)** or zero-coupon bond is one party borrow and another pay back with interest
  * **[cfd](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli/examples/cfd)** or contract for differences is where two parties settle for price differences
  * **[covered call](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli/examples/coveredCall)** transfers a token if the counter-party exercises the option
  * **[escrow](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli/examples/escrow)** purchase of an item at a price between buyer and seller with a mediator
  * **[simple](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli/examples/simple)** takes as deposit, waits for a notification, makes a payment
  * **[swap](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli/examples/swap)** swaps native tokens between two parties
  
This notebook will use a custom build **swap** contract.  

A swap smart contract in Cardano allows users to exchange one cryptocurrency for another  
without the need for a centralized exchange.  
The basic idea is that two parties can agree to exchange different amounts of two different cryptocurrencies,  
and the smart contract automatically executes the trade when both parties have fulfilled their end of the agreement.  

## Installation

### versions

In [18]:
# installed modules in python
conda_version = ! conda -V
print(f"conda       : {conda_version[0].split()[1]}")
pip_version = ! pip -V
print(f"pip         : {pip_version[0].split()[1]}")
python_version = ! python -V
print(f"python      : {python_version[0].split()[1]}")
cardano_version = ! pip list |grep -i cardano
print(f"cardano     : {cardano_version[0].split()[1]}")
pandas_version = ! pip list |grep -i pandas
print(f"pandas      : {pandas_version[0].split()[1]}")
numpy_version = ! pip list |grep -i numpy
print(f"numpy       : {numpy_version[0].split()[1]}")

conda       : 4.12.0
pip         : 21.2.4
python      : 3.9.12
cardano     : 0.8.2
pandas      : 1.4.3
numpy       : 1.23.0
json        : 0.1.13


### modules

In [45]:
# used for loading a webpage
from IPython.display import Javascript
from IPython.core.display import HTML

import re           # regex tool
import pandas as pd # tabular data tool
import numpy as np  # needed for helper function
import json         # json data structure tool

# environment packages
import platform
import psutil
import os

# https://stackabuse.com/executing-shell-commands-with-python/
import subprocess

# pd.describe_option('display')            # show all pandas options, parameters can slow down notebook
pd.set_option('display.max_colwidth', 80)  # default 50, the maximum width in characters of a column
pd.set_option('display.max_columns', 20)   # default 20, the maximum amount of columns in view 
pd.set_option('display.max_rows', 10)      # default 60, the maximum amount of rows in view

In [46]:
# setup a mouse over tooltip feature
def _set_css_style(css_file_path):
   """
   Read the custom CSS file and load it into Jupyter.
   Pass the file path to the CSS file.
   """
   styles = open(css_file_path, "r").read()
   st = '<style>%s</style>' % styles     
   return HTML(st)

_set_css_style('../scripts/tooltip_style.css')

<div class="ttooltip">Marlowe<span class="tooltiptext">Tooltip text</span></div>

### server
You start with **Ubuntu** on a virtual machine in the cloud.

In [37]:
# Virtual Machine environment 
print(f"Cores : {psutil.cpu_count(logical=True)} ({psutil.cpu_freq().current/1000:.0f}GHz)")
print(f"Memory: {psutil.virtual_memory().total/(1024**3):.2f} GB ({psutil.virtual_memory().percent:.0f}% used)")
print(f"Swap  : {os.path.getsize('/swapfile')/(1024**3):.0f} GB")
disk_size = psutil.disk_usage(psutil.disk_partitions()[0].mountpoint).total
disk_used = psutil.disk_usage(psutil.disk_partitions()[0].mountpoint).percent
disk_fs   = psutil.disk_partitions()[0].fstype 
print(f"Disk  : {disk_size/(1024**3):.0f} GB {disk_fs} ({disk_used:.0f}% used)")
print(f"System: {platform.uname().version.split('~')[1].split()[0]}")

Cores : 8 (3GHz)
Memory: 62.81 GB (8% used)
Swap  : 8 GB
Disk  : 242 GB ext4 (65% used)
System: 20.04.1-Ubuntu


### software
Connect to your VM using a Terminal (Putty).  
**Update** your installation.  
Install **curl** and **rsync**.  
```sh
sudo sh -c 'apt update && apt install curl rsync'
```

**Install nixos** from the official installation script.  
Nixos is a form of virtual environment like docker with cryptographic hashed package manager Nix to eliminate dependency hell.
```sh
sh <(curl -L https://nixos.org/nix/install) --daemon
```

Add content to **nix.conf** and **restart** the nix-daemon.  
These lines allow for a fast build of nixos, downloading trusted binaries instead of compiling from scratch.  
```sh
sudo sh -c "echo 'substituters = https://hydra.iohk.io https://iohk.cachix.org https://cache.nixos.org/' >> /etc/nix/nix.conf"
sudo sh -c "echo 'trusted-public-keys = hydra.iohk.io:f/Ea+s+dFdN+3Y/G+FDgSq+a5NEWhJGzdjvKNGv0/EQ= iohk.cachix.org-1:DpRUyj7h7V830dp/i6Nti+NEO2/nhblbov/8MW7Rqoo= cache.nixos.org-1:6NCHdD59X431o0gWypbMrAURkbJ16ZPMQFGspcDShjY=' >> /etc/nix/nix.conf"
sudo sh -c "echo 'experimental-features = nix-command' >> /etc/nix/nix.conf"  
sudo sh -c "echo 'extra-experimental-features = flakes' >> /etc/nix/nix.conf"  
sudo sh -c 'echo "trusted-users = $0" >> /etc/nix/nix.conf' `whoami`
```

In [5]:
# here is the content of the config file
!cat /etc/nix/nix.conf

build-users-group = nixbld
substituters = https://hydra.iohk.io https://iohk.cachix.org https://cache.nixos.org/
trusted-public-keys = hydra.iohk.io:f/Ea+s+dFdN+3Y/G+FDgSq+a5NEWhJGzdjvKNGv0/EQ= iohk.cachix.org-1:DpRUyj7h7V830dp/i6Nti+NEO2/nhblbov/8MW7Rqoo= cache.nixos.org-1:6NCHdD59X431o0gWypbMrAURkbJ16ZPMQFGspcDShjY=
trusted-users = ubuntu
experimental-features = nix-command
extra-experimental-features = flakes


Restart Nix  
```sh
sudo systemctl restart nix-daemon.service
```

In [6]:
# here is the version of Nix
!/nix/var/nix/profiles/default/bin/nix --version

nix (Nix) 2.12.0


### install marlowe-cli

1. Download
```sh
git clone https://github.com/input-output-hk/marlowe-cardano.git
```
2. Change directory
```sh
cd marlowe-cardano/marlowe-cli
```
2. Compile (be patient ...)
```sh
nix develop
```

### install cardano-node

1. Create directory


1. Download
```sh
git clone https://github.com/input-output-hk/marlowe-cardano.git
```
2. Change directory
```sh
cd marlowe-cardano/marlowe-cli
```
2. Compile (be patient ...)
```sh
nix develop
```

In [ ]:
https://github.com/input-output-hk/PROJECT/releases/latest/download/package.zip

In [ ]:
download/install **cardano-cli** executables:  

Download the latest release of 'cardano-node' from the GitHub releases page.  
create directory, Extract, clean up.  
```sh
mkdir ~/cardano
cd ~/cardano
wget https://update-cardano-mainnet.iohk.io/cardano-node-releases/cardano-node-1.35.4-linux.tar.gz  
tar xzf cardano-node-1.35.4-linux.tar.gz  
rm cardano-node-1.35.4-linux.tar.gz  
```
---
download/install **cardano-wallet** executables:  

Download the latest release of 'cardano-wallet' from the GitHub releases page.  
Extract, delete, move directory, clean up.  
```sh
cd ~/cardano
wget https://github.com/input-output-hk/cardano-wallet/releases/download/v2022-12-14/cardano-wallet-v2022-12-14-linux64.tar.gz
tar xzf cardano-wallet-v2022-12-14-linux64.tar.gz
rm cardano-wallet-v2022-12-14-linux64.tar.gz
mv cardano-wallet-v2022-12-14-linux64/* ~/cardano/
rm cardano-wallet-v2022-12-14-linux64
```
---
download/install **marlowe-cli** executables:  
  
at this stage you can only build/compile with **nix develop**,  
we can copy this binary file from /nix/store/...  
```sh
cp /nix/store/*-marlowe-cli-exe-marlowe-cli-*/bin/marlowe-cli ~/cardano  
```
---
add the ~/cardano directory to your $PATH variable,  
so that you can execute the binary files in that directory from the command line.  
To make this persistent across reboots, we add the export command to the shell initialization file.  
```sh
sudo sh -c "echo 'export PATH=$PATH:$HOME/cardano' >> $HOME/.bashrc"
```

re-execute the shell initialization files and apply the updated settings.  
```sh
source ~/.bashrc
```

### custom helper functions

helper functions transform transaction status into a tabular form.   

be aware to update the magic='...' value,  
depending on the Cardano Blockchain network of choice  

'1567' means marlowe-pioneer  
'2' means preview  
'1' means preprod  
'764824073' means mainnet

In [4]:
def query_df(address, magic='2', cardanoCLI="/home/ubuntu/cardano/cardano-cli"):
    '''
    query_df(address, magic, cardanoCLI)
    
    address : (str) [required] is the wallet address you want to check for last transaction state
    magic : (str) [optional] is the key used to identify the mainnet, testnet, other networks
    cardanoCLI : (str) [optional] the binary file of the cardano-cli on your system
    '''

    def hex_to_ascii(s):
        # small function to decode hex strings to ascii (translation)
        try:
            return bytearray.fromhex(s).decode()
        except ValueError:
            return None

    # shell command, execute and store result in a temporary json file
    cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", magic, "--address", address, "--out-file", 'address.json']
    result = subprocess.run(cmd, capture_output=True, text=True)
    # load this json file into a dataframe
    df = pd.read_json('address.json', orient='index').reset_index()

    # test if empty query, then return empty line with address
    if len(df.columns) < 2:
        # close off
        return pd.DataFrame({'address':[address], 'TxHash':np.nan, 'TxIx':0, 'lovelace':0, 'TokenPolicyId':np.nan, 'TokenNameHex':np.nan, 'TokenNameAscii':np.nan, 'TokenAmount':0, 'datumhash':np.nan})

    # replace feature names with interpretable naming (others are left as-is) 
    df = df.rename(columns={'index': 'TxHash',
                            'datumhash': 'datumhash',
                            'address': 'address',
                            'value': 'value'})
    # split the # value into a seperate column and remove the original
    df = df.join(df['TxHash'].str.split('#', regex=False, expand=True), how='left')
    df = df.drop(['TxHash'], axis=1)
    # replace feature names with interpretable naming (others are left as-is) 
    df = df.rename(columns={0:'TxHash', 1: 'TxIx'})
    # normalize the value column ex:{lovelace:3000000} into 2 new columns and remove original column
    df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
    # change data type from float to integer
    df['lovelace'] = df['lovelace'].fillna(0)
    df['TxIx'] = df['TxIx'].fillna(0)
    df['lovelace'] = df['lovelace'].astype('int')
    df['TxIx'] = df['TxIx'].astype('int')
    # drop all columns with all values NA
    df = df.dropna(axis=1, how='all')
    
    # if the last column name word size is 56 characters wide:
    # try if this is needed, if no data is present then skip
    try:
        # unselect all known columns, keep last ones column name
        cn = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace'])]].iloc[:, -1].name
        # check if this name is 64 bytes long ? 
        if len(cn) == 56:
            # make a copy into a new column name
            df['TokenPolicyId'] = df[cn]
            # write the values 'column name' only on True state locations
            df['TokenPolicyId'] = df['TokenPolicyId'].notna().replace({True: cn, False: np.nan})
            
            # normalize the value column ex:{}
            df = pd.concat([df.drop([cn], axis=1), df[cn].apply(pd.Series)], axis=1)
            # drop all columns with all values NA
            df = df.dropna(axis=1, how='all')
            
            # build a secondary dataframe to be concatenated at the end with the hextokenid and tokenamount
            # exclude all known columns
            two = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId'])]]
            # stack the column name and values into one dataframe and reset the index
            tmp = two.stack().apply(pd.Series).reset_index()
            # set the index to this column values, later for merging with df
            tmp = tmp.set_index('level_0')
            # rename these 2 columns
            tmp.columns = ['TokenNameHex', 'TokenAmount']
            # merge
            df = pd.concat([df, tmp], axis=1)
            # set tokenamount to integer dtype
            df['TokenAmount'] = df['TokenAmount'].fillna(0)
            df['TokenAmount'] = df['TokenAmount'].astype('int')
            # remove the bogus columns, clean up
            df = df.drop(tmp['TokenNameHex'], axis=1)
            # add a column with the ascii transation
            df['TokenNameAscii'] = df['TokenNameHex'].astype('str').apply(hex_to_ascii)
            # add empty column data so we can combine all dataframes into one later
            df['datumhash'] = np.nan
            # structure in chronologic order the column names
            df = df[['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId', 'TokenNameHex', 'TokenNameAscii', 'TokenAmount', 'datumhash']]
            return df
            
    # in the case of no data present add these extra empty columns
    # in fact this will never occur, below code... 
    except:
        df['datumhash'] = np.nan
        df['TokenPolicyId'] = np.nan
        df['TokenNameHex'] = np.nan
        df['TokenAmount'] = 0
        return df

    # add empty column data so we can combine all dataframes into one later
    df['TokenPolicyId'] = np.nan
    df['TokenNameHex'] = np.nan
    df['TokenNameAscii'] = np.nan
    df['TokenAmount'] = 0
    # structure in chronologic order the column names
    df = df[['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId', 'TokenNameHex', 'TokenNameAscii', 'TokenAmount', 'datumhash']]
    return df

## Check

### find binaries
Here we search for the binaries installed in nix-shell.  

  * **cardano-cli** used for generating keys, constructing transactions, creating certificates on the cardano node running in the background
  * **marlowe-cli** used for serialising Marlowe contracts to validators, datums, redeemers and also computes hashes and addresses with cardano-cli
  * **cardano-wallet** used for sending and receiving payments, list, create, update, or delete wallets  
  
  

In [9]:
# !sudo find /nix/store -wholename '*/bin/cardano-cli'

In [128]:
!sudo find /nix/store -wholename '*/bin/marlowe-cli'

/nix/store/s2jwd3gmga4jw4yw3vpnahw65p7j9z6j-marlowe-cli-exe-marlowe-cli-0.0.10.0/bin/marlowe-cli
/nix/store/4hvi690affn9p4qfz508cixnmlqfscqf-marlowe-cli-exe-marlowe-cli-0.0.8.1/bin/marlowe-cli


git clone cardano-wallet...  
go into it and run with Nix to build  
nix-build -A cardano-wallet  
then wait 4-5 hours

In [11]:
# !sudo find /nix/store -wholename '*/bin/cardano-wallet'

download/install **cardano-cli** executables:  

Download the latest release of 'cardano-node' from the GitHub releases page.  
create directory, Extract, clean up.  
```sh
mkdir ~/cardano
cd ~/cardano
wget https://update-cardano-mainnet.iohk.io/cardano-node-releases/cardano-node-1.35.4-linux.tar.gz  
tar xzf cardano-node-1.35.4-linux.tar.gz  
rm cardano-node-1.35.4-linux.tar.gz  
```
---
download/install **cardano-wallet** executables:  

Download the latest release of 'cardano-wallet' from the GitHub releases page.  
Extract, delete, move directory, clean up.  
```sh
cd ~/cardano
wget https://github.com/input-output-hk/cardano-wallet/releases/download/v2022-12-14/cardano-wallet-v2022-12-14-linux64.tar.gz
tar xzf cardano-wallet-v2022-12-14-linux64.tar.gz
rm cardano-wallet-v2022-12-14-linux64.tar.gz
mv cardano-wallet-v2022-12-14-linux64/* ~/cardano/
rm cardano-wallet-v2022-12-14-linux64
```
---
download/install **marlowe-cli** executables:  
  
at this stage you can only build/compile with **nix develop**,  
we can copy this binary file from /nix/store/...  
```sh
cp /nix/store/*-marlowe-cli-exe-marlowe-cli-*/bin/marlowe-cli ~/cardano  
```
---
add the ~/cardano directory to your $PATH variable,  
so that you can execute the binary files in that directory from the command line.  
To make this persistent across reboots, we add the export command to the shell initialization file.  
```sh
sudo sh -c "echo 'export PATH=$PATH:$HOME/cardano' >> $HOME/.bashrc"
```

re-execute the shell initialization files and apply the updated settings.  
```sh
source ~/.bashrc
```

In [8]:
# list all the binaries needed to create and manage Cardano addresses,
# running a Cardano node, or interacting with the Cardano network using the CLI
!ls -l ~/cardano

total 455296
-rw-rw-r-- 1 ubuntu ubuntu     9459 Dec 29 16:41 alonzo-genesis.json
drwxr-xr-x 5 ubuntu ubuntu     4096 Dec 14 18:38 auto-completion
-rwxr-xr-x 1 ubuntu ubuntu  4502992 Dec 14 18:38 bech32
-rw-rw-r-- 1 ubuntu ubuntu     5607 Dec 29 16:41 byron-genesis.json
-rwxr-xr-x 1 ubuntu ubuntu  8601960 Dec 14 18:38 cardano-address
-rwxr-xr-x 1 ubuntu ubuntu 40594416 Dec 14 18:38 cardano-cli
-rwxr-xr-x 1 ubuntu ubuntu 51948112 Dec 14 18:38 cardano-node
-rwxr-xr-x 1 ubuntu ubuntu 24962016 Dec  2 16:03 cardano-node-chairman
-rwxr-xr-x 1 ubuntu ubuntu  5323528 Dec  2 16:03 cardano-ping
-rwxr-xr-x 1 ubuntu ubuntu 24780800 Dec  2 16:03 cardano-submit-api
-rwxr-xr-x 1 ubuntu ubuntu  6764576 Dec  2 16:03 cardano-testnet
-rwxr-xr-x 1 ubuntu ubuntu  3497856 Dec  2 16:03 cardano-topology
-rwxr-xr-x 1 ubuntu ubuntu 48705536 Dec 14 18:38 cardano-wallet
-rwxr-xr-x 1 ubuntu ubuntu 22677600 Dec  2 16:03 chain-sync-client-with-ledger-state
-rw-rw-r-- 1 ubuntu ubuntu     3072 Dec 29 16:41 config.json

In [5]:
# Add the '~/cardano' directory to the PATH environment variable
os.environ['PATH'] = os.environ['PATH'] + ':' + os.path.expanduser('~/cardano')

# check the result, call the environment variable from within jupyter notebook
!echo $PATH

/anaconda/envs/py39_cardano/bin:/anaconda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ubuntu/cardano


In [5]:
# !echo $PATH

/anaconda/envs/py39_cardano/bin:/anaconda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


cardano-cli 1.35.4 - linux-x86_64 - ghc-8.10
git rev ebc7be471b30e5931b35f9bbc236d21c375b91bb


marlowe-cli 0.0.10.0


In [17]:
!cardano-wallet version

v2022-12-14 (git revision: bbf11d4feefd5b770fb36717ec5c4c5c112aca87)


In [7]:
# !sudo find /nix/store -wholename '*/bin/cardano-cli'
# !sudo find /nix/store -wholename '*/bin/marlowe-cli'
# !sudo find /nix/store -wholename '*/bin/cardano-wallet'

/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli
/nix/store/9ijr3m1g4rr487q8gz35h3d27bfrxkyw-marlowe-cli-exe-marlowe-cli-0.0.4.4/bin/marlowe-cli
/nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet


### save binary path

In [6]:
# cardanoCLI = "/nix/store/6ykddplws1py6797j4paq93bdr05x5j3-cardano-cli-exe-cardano-cli-1.35.3/bin/cardano-cli"
cardanoCLI    = "/home/ubuntu/cardano/cardano-cli"
marloweCLI    = "/home/ubuntu/cardano/marlowe-cli"
cardanoWALLET = "/home/ubuntu/cardano/cardano-wallet"

# cardanoCLI    = "~/cardano/cardano-cli"
# marloweCLI    = "~/cardano/marlowe-cli"
# cardanoWALLET = "~/cardano/cardano-wallet"

In [8]:
# cardanoCLI = "/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli"
# marloweCLI = "/nix/store/9ijr3m1g4rr487q8gz35h3d27bfrxkyw-marlowe-cli-exe-marlowe-cli-0.0.4.4/bin/marlowe-cli"
# cardanoWALLET = "/nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet"

### check versions

latest version available:  
  * CLI tools:
    * [cardano-cli](https://github.com/input-output-hk/cardano-node/releases/latest)
    * [marlowe-cli](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli)
    * [cardano-wallet](https://github.com/input-output-hk/cardano-wallet/releases/latest)
  * services:
    * cardano-node run
    * cardano-wallet serve 

In [10]:
!cardano-cli --version

cardano-cli 1.35.4 - linux-x86_64 - ghc-8.10
git rev ebc7be471b30e5931b35f9bbc236d21c375b91bb


In [7]:
!marlowe-cli --version

marlowe-cli 0.0.10.0


In [12]:
!cardano-wallet version

v2022-12-14 (git revision: bbf11d4feefd5b770fb36717ec5c4c5c112aca87)


In [13]:
!cardano-node --version

cardano-node 1.35.4 - linux-x86_64 - ghc-8.10
git rev ebc7be471b30e5931b35f9bbc236d21c375b91bb


In [14]:
# ! /nix/store/6ykddplws1py6797j4paq93bdr05x5j3-cardano-cli-exe-cardano-cli-1.35.3/bin/cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [9]:
# !/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli --version

cardano-cli 1.34.1 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [15]:
# ! /nix/store/s2jwd3gmga4jw4yw3vpnahw65p7j9z6j-marlowe-cli-exe-marlowe-cli-0.0.10.0/bin/marlowe-cli --version

marlowe-cli 0.0.10.0


In [10]:
# !/nix/store/9ijr3m1g4rr487q8gz35h3d27bfrxkyw-marlowe-cli-exe-marlowe-cli-0.0.4.4/bin/marlowe-cli --version

marlowe-cli 0.0.4.4


In [34]:
# ! /home/ubuntu/notebooks/cardano/cardano-wallet-v2022-12-14-linux64/cardano-wallet version

v2022-12-14 (git revision: bbf11d4feefd5b770fb36717ec5c4c5c112aca87)


In [11]:
# !/nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet version

v2022-01-18 (git revision: 0000000000000000000000000000000000000000)


### start node services

This needs to be updated later to include tmux script here...  
but I want one window running 'cardano-node'  
and another running 'cardano-wallet'  
and then a third one for htop  
and maybe a fourth one for regular user shell inputs  

[**cardano-node** howto](https://developers.cardano.org/docs/get-started/running-cardano/)  
download the blockchain network configuration files for your specific NetworkMagic: 2  
```sh
cd ~/cardano
curl -O -J https://book.world.dev.cardano.org/environments/preview/config.json
curl -O -J https://book.world.dev.cardano.org/environments/preview/db-sync-config.json
curl -O -J https://book.world.dev.cardano.org/environments/preview/submit-api-config.json
curl -O -J https://book.world.dev.cardano.org/environments/preview/topology.json
curl -O -J https://book.world.dev.cardano.org/environments/preview/byron-genesis.json
curl -O -J https://book.world.dev.cardano.org/environments/preview/shelley-genesis.json
curl -O -J https://book.world.dev.cardano.org/environments/preview/alonzo-genesis.json
```

start the node (listen on all connections)
```sh
cd ~/cardano
cardano-node run \
   --topology topology.json \
   --database-path ~/cardano/db/ \
   --socket-path /tmp/node.socket \
   --host-addr 0.0.0.0 \
   --config config.json
```

---

[**cardano-wallet** howto](https://input-output-hk.github.io/cardano-wallet/user-guide/common-use-cases/how-to-start-wallet-server)  
start a wallet server (listen on all connections)
```sh
cd ~/cardano
cardano-wallet serve \
   --listen-address 0.0.0.0 \
   --port 8090 \
   --node-socket /tmp/node.socket \
   --testnet byron-genesis.json \
   --database ~/cardano/wallet-db \
   --token-metadata-server https://metadata.cardano-testnet.iohkdev.io
```

#### option: tips and tricks for using tmux

Launch a **tmux** session to **start** the blockchain node and wallet services.  
Wait, let it synchronise...  
tmux, Terminal multiplexer for running multiple terminal session windows.  

```sh
code/scripts/cardano_start.sh
```  

Here are some commands to use tmux:  

    PANE commands
    ctrl+b ->    to move to the right pane, up, down and left arrow also possible
    ctrl+b z     enlarge or zoom to one pane
    ctrl+b "     for split horizontal
    ctrl+b %     for split vertikal
    ctrl+b d     to kill a screen pane
    
    WINDOW commands
    ctrl+b c     create new window
    ctrl+b p     previous window
    ctrl+b n     next window
    ctrl+b &     kill a window
    
    SESSION commands
    tmux list-sessions    view a list of all available sessions
    tmux ls               (short version)
    tmux attach           previous last used tmux session
    tmux a                (short version)
    tmux attach -t 1      open tmux session id 1

To enable scroll mode by default when you start tmux,  
you can add the following line to your tmux configuration file  
```sh
sudo sh -c "echo 'set -g mouse on' >> $HOME/.tmux.conf"
```

re-execute the shell initialization files and apply the updated settings.  
```sh
source ~/.bashrc
```

### node.socket
The cardano-node uses IPC (Inter-Process-Communication) for communicating with other Cardano components.  
Components like cardano-cli, cardano-wallet, marlowe-cli and cardano-db-sync.  
So, basically this file helps connect your CLI instance to the active cardano-node currently syncing the blockchain.  

In [14]:
!date
!sudo ls -al /tmp/node.socket

Thu Jan  5 14:51:40 CET 2023
srwxrwxr-x 1 ubuntu ubuntu 0 Jan  5 14:49 /tmp/node.socket


add /tmp/node.socket into the environment variable and make it persistent  
```sh
sudo sh -c "echo 'export CARDANO_NODE_SOCKET_PATH="/tmp/node.socket"' >> $HOME/.bashrc"
```
re-execute the shell initialization files and apply the updated settings.  
```sh
source ~/.bashrc
```

In [8]:
# this is only needed for cardano-cli, not marlowe-cli
os.environ['CARDANO_NODE_SOCKET_PATH']="/tmp/node.socket"
# check the result, call the environment variable from within jupyter notebook
!echo $CARDANO_NODE_SOCKET_PATH

/tmp/node.socket


## Wallets

### sync

Be patient and wait for the cardano-node in nix-shell to download and **synchronize the blockchain history.**  
This can take many hours.  

After sync, the cardano-wallet can be used either through  
HTTP Application Programming Interface (**API**) and command-line interface (**CLI**)  

  * [Python Cardano module](https://github.com/emesik/cardano-python)

**[python module for cardano-wallet working on mainnet 'notebook'](./howto_cardano-wallet.ipynb)**

## wallet

### generate

mnemonic seed phrase:  

  * 12 words (Byron legacy wallet)
  * 15 words (Incentivised Testnet Rewards wallet)
  * **24 words** (Shelley wallet)
  * 27 words (Byron legacy paper wallet)
  
generate new Shelley wallet  
```sh
cardano-wallet recovery-phrase generate --size 24
```
generated example output:
```sh
movie use luxury dumb kingdom garden mammal strong laundry weasel legal mesh movie old cancel paddle color pluck slight wheel mouse front slice pupil
```

[**how secure are seed phrases**](https://www.reddit.com/r/CryptoCurrency/comments/w0bv37/how_secure_are_seed_phrases)  

Convert the seed phrase into a root private key 'wallet1.prv'.  
```sh
cat wallet1.seed | cardano-wallet key from-recovery-phrase Shelley | cardano-wallet key child 1852H/1815H/0H/0/0 > wallet1.prv
```

Generate the *.**skey**, *.**vkey** and *.**address** files (using *.prv)
```sh
cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file wallet1.prv --out-file wallet1.skey
cardano-cli key verification-key --signing-key-file wallet1.skey --verification-key-file wallet1.vkey
cardano-cli address build --testnet-magic 1567 --payment-verification-key-file wallet1.vkey > wallet1.addr
```

**Question:** I don't understand why the size of the public address is smaller than a normal cardano-wallet address ?  
**Question:** I don't understand why we need a pkey, skey, vkey and what they mean ?  
**Question:** I don't understand why we need these files for marlowe-cli transactions and how do we use a light wallet, web wallet instead of these skey, vkey ?  
**Question:** what is the purpose of the vkey (=verification key) ?  
**Question:** what is the purpose of the skey (=signing key) ?  
**Question:** is the private key *.prv somehow the translation from the seed phrase to a different form, each word representing 4 digit number ?  

READ THIS: https://developers.cardano.org/docs/integrate-cardano/creating-wallet-faucet#creating-a-wallet

#### wallet 1

In [9]:
# generate 24 word key phrase
cmd = [cardanoWALLET, "recovery-phrase", "generate", "--size", str(24)]  # cardano-wallet CLI command to generate NEW mneumic seed phrase with 24 words
result = subprocess.run(cmd, capture_output=True, text=True)             # run the command
print("wallet 1 seed phrase:\n\n", result.stdout)                        # demonstrate the result
open('wallet1.seed', 'w').write(result.stdout)                           # save the result to a file 'wallet1.seed'

wallet 1 seed phrase:

 globe property diary express globe witness they govern can argue occur behind leopard box that visa dust around web hour fatigue asset blur produce



148

In [10]:
# generate the address, skey and vkey files
!cat wallet1.seed | cardano-wallet key from-recovery-phrase Shelley | cardano-wallet key child 1852H/1815H/0H/0/0 > wallet1.prv
!cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file wallet1.prv --out-file wallet1.skey
!cardano-cli key verification-key --signing-key-file wallet1.skey --verification-key-file wallet1.vkey
!cardano-cli address build --testnet-magic 2 --payment-verification-key-file wallet1.vkey > wallet1.address

In [19]:
# # generate the address, skey and vkey files
# !cat wallet1.seed | /nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet key from-recovery-phrase Shelley | /nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet key child 1852H/1815H/0H/0/0 > wallet1.prv
# !/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file wallet1.prv --out-file wallet1.skey
# !/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli key verification-key --signing-key-file wallet1.skey --verification-key-file wallet1.vkey
# !/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli address build --testnet-magic 1567 --payment-verification-key-file wallet1.vkey > wallet1.address

In [19]:
!cat wallet1.prv

addr_xsk17p2rdlx32gsqjkt5ay6t3hj72pazppkcj5jezcv59p49vmvnm9f0waaqxaaf79kjlx5eqv345c6p7xsnztd7f4f2x6ptcrxc2n4gsajljlsfccgujguezgwxc97f6y39klu09yrle4wpsq45ustefzxdygddmldr

In [18]:
# !cat wallet1.prv

addr_xsk1wptvk7ck44jjsng8nxd0ukwht2rq9qfwse8ugfsch8ytjqkfc3xsc7s7pegvzq084l5v5dfy0g49e238rzqzjt9lgshrqcpwgj4uf6fcqfa7gqt6cu0dg5zmhhqt4995zg3uvk8cldv2m0fvsccj5l726ursatrr

In [20]:
!cat wallet1.skey

{
    "type": "PaymentExtendedSigningKeyShelley_ed25519_bip32",
    "description": "",
    "cborHex": "5880f05436fcd15220095974e934b8de5e507a2086d89525916194286a566d93d952f777a0377a9f16d2f9a9903235a6341f1a1312dbe4d52a3682bc0cd854ea88769fa8121913eb89d3e63ad827327209723479ec9c586564e7c9a3d5bf50192b745f97e09c611c92399121c6c17c9d1225b7f8f2907fcd5c1802b4e4179488cd22"
}


In [19]:
# !cat wallet1.skey

{
    "type": "PaymentExtendedSigningKeyShelley_ed25519_bip32",
    "description": "",
    "cborHex": "58807056cb7b16ad65284d07999afe59d75a8602812e864fc42618b9c8b902c9c44d0c7a1e0e50c101e7afe8ca35247a2a5caa271880292cbf442e30602e44abc4e97060d53791200399a29871511d5f35bd8fd06cfb177c54fd4886c4edffa5c78938027be4017ac71ed4505bbdc0ba94b41223c658f8fb58adbd2c86312a7fcad7"
}


In [21]:
!cat wallet1.vkey

{
    "type": "PaymentExtendedVerificationKeyShelley_ed25519_bip32",
    "description": "",
    "cborHex": "58409fa8121913eb89d3e63ad827327209723479ec9c586564e7c9a3d5bf50192b745f97e09c611c92399121c6c17c9d1225b7f8f2907fcd5c1802b4e4179488cd22"
}


In [20]:
# !cat wallet1.vkey

{
    "type": "PaymentExtendedVerificationKeyShelley_ed25519_bip32",
    "description": "",
    "cborHex": "58407060d53791200399a29871511d5f35bd8fd06cfb177c54fd4886c4edffa5c78938027be4017ac71ed4505bbdc0ba94b41223c658f8fb58adbd2c86312a7fcad7"
}


In [22]:
!cat wallet1.address

addr_test1vqhrmz3u5sawlys6tfctctqv78c5d8k4x6reac0l992m46cd95fee

In [21]:
# !cat wallet1.address

addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t

#### wallet 2

In [11]:
# generate 24 word key phrase
cmd = [cardanoWALLET, "recovery-phrase", "generate", "--size", "24"]
result = subprocess.run(cmd, capture_output=True, text=True)
print("wallet 2 seed phrase:\n\n", result.stdout)
open('wallet2.seed', 'w').write(result.stdout)

wallet 2 seed phrase:

 whale evil fabric scene unable path level wing favorite frog prefer hundred coffee display uncover super rigid perfect notice gauge outside labor venture aware



160

In [12]:
!cat wallet2.seed | cardano-wallet key from-recovery-phrase Shelley | cardano-wallet key child 1852H/1815H/0H/0/0 > wallet2.prv
!cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file wallet2.prv --out-file wallet2.skey
!cardano-cli key verification-key --signing-key-file wallet2.skey --verification-key-file wallet2.vkey
!cardano-cli address build --testnet-magic 2 --payment-verification-key-file wallet2.vkey > wallet2.address

In [25]:
!cat wallet2.address

addr_test1vrwycxp7gx8gcf2wgzkkpuptrpe55ejlrcyntfc0yz6j75qjal5w7

In [28]:
# !cat wallet2.address

addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6

In [26]:
# list all the wallet files generated
!date
!ls -al wallet*

Thu Jan  5 08:20:32 CET 2023
-rw-rw-r-- 1 ubuntu ubuntu  63 Jan  5 08:20 wallet1.address
-rw-rw-r-- 1 ubuntu ubuntu 169 Jan  5 08:20 wallet1.prv
-rw-rw-r-- 1 ubuntu ubuntu 154 Jan  5 08:20 wallet1.seed
-rw------- 1 ubuntu ubuntu 367 Jan  5 08:20 wallet1.skey
-rw------- 1 ubuntu ubuntu 244 Jan  5 08:20 wallet1.vkey
-rw-rw-r-- 1 ubuntu ubuntu  63 Jan  5 08:20 wallet2.address
-rw-rw-r-- 1 ubuntu ubuntu 169 Jan  5 08:20 wallet2.prv
-rw-rw-r-- 1 ubuntu ubuntu 154 Jan  5 08:20 wallet2.seed
-rw------- 1 ubuntu ubuntu 367 Jan  5 08:20 wallet2.skey
-rw------- 1 ubuntu ubuntu 244 Jan  5 08:20 wallet2.vkey


## Contract

### design
We can develop a smart contract in different ways, javascript, haskell, marlowe and more in the future.  
This example was written by Marco in javascript (.js) and can be [**found here.**](https://gist.github.com/marco-martins/670832702aaa2c9258cfecef5a1b7fa3)  
The idea here is to have 2 wallet users each sending a certain amount in ADA  
and then each receives what the other person sended, called a swap.  
Consult the [**marlowe playground**](https://marlowe-playground-marlowe-pioneers.plutus.aws.iohkdev.io/#/) for editing and simulating how this works.  
![marlowe playground javascript code replacement](../../image/marlowe_playground_swap_javascript.png)

Alternatively you can use the following **Marlowe** code (also found after compilation).  

```sh
When
    [Case
        (Deposit
            (Role "Seller")
            (Role "Seller")
            (Token "" "")
            (ConstantParam "Seller Amount")
        )
        (When
            [Case
                (Deposit
                    (Role "Buyer")
                    (Role "Buyer")
                    (Token "" "")
                    (ConstantParam "Buyer Amount")
                )
                (Pay
                    (Role "Seller")
                    (Party (Role "Buyer"))
                    (Token "" "")
                    (ConstantParam "Seller Amount")
                    (Pay
                        (Role "Buyer")
                        (Party (Role "Seller"))
                        (Token "" "")
                        (ConstantParam "Buyer Amount")
                        Close 
                    )
                )]
            (TimeParam "Deadline")
            Close 
        )]
    (TimeParam "Deadline")
    Close 
```

You can then also view as blocks and code visually using **blockly**.  
  
![marlowe playground blockly code](../../image/marlowe_playground_swap_blockly.png)  


**You can export the script to ".json"**  
We stored this script and it's content looks like this

In [20]:
!cat ../../data/contracts/swap_contract.json

{"when":[{"then":{"when":[{"then":{"token":{"token_name":"","currency_symbol":""},"to":{"party":{"role_token":"Buyer"}},"then":{"token":{"token_name":"","currency_symbol":""},"to":{"party":{"role_token":"Seller"}},"then":"close","pay":0,"from_account":{"role_token":"Buyer"}},"pay":0,"from_account":{"role_token":"Seller"}},"case":{"party":{"role_token":"Buyer"},"of_token":{"token_name":"","currency_symbol":""},"into_account":{"role_token":"Buyer"},"deposits":0}}],"timeout_continuation":"close","timeout":1658230520814},"case":{"party":{"role_token":"Seller"},"of_token":{"token_name":"","currency_symbol":""},"into_account":{"role_token":"Seller"},"deposits":0}}],"timeout_continuation":"close","timeout":1658230520814}

In [21]:
with open('../../data/contracts/swap_contract.json', 'r') as file:
    parsed = json.load(file)
    print(json.dumps(parsed, indent=2))

{
  "when": [
    {
      "then": {
        "when": [
          {
            "then": {
              "token": {
                "token_name": "",
                "currency_symbol": ""
              },
              "to": {
                "party": {
                  "role_token": "Buyer"
                }
              },
              "then": {
                "token": {
                  "token_name": "",
                  "currency_symbol": ""
                },
                "to": {
                  "party": {
                    "role_token": "Seller"
                  }
                },
                "then": "close",
                "pay": 0,
                "from_account": {
                  "role_token": "Buyer"
                }
              },
              "pay": 0,
              "from_account": {
                "role_token": "Seller"
              }
            },
            "case": {
              "party": {
                "role_token": "Buyer"
              

### parameters
The **parties agree beforehand to define certain parameter values** to be used for the contract script.  
Notice that the values can only be **integers** to be precise when used on chain.  
This is to remove machine independable float value discrepancies.  
So, even time is translated to an integer representation for this reason,  
so is the value of ADA currently having 5 decimal points called lovelaces.  
The rest is often represented as **string** values.  

In [ ]:
# Here we need extra work on the TIP and the deadline timings

In [13]:
MAGIC                 = '2'          # marlowe-pioneers testnet ID = 1567 (provided by developers)
SOCKET                = '/tmp/node.socket'

In [14]:
cmd = [marloweCLI, "util", "slotting", 
       "--testnet-magic", MAGIC,
       "--socket-path", SOCKET]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

{
    "scSlotLength": 1000,
    "scSlotZeroTime": 1666656000000
}




In [15]:
SLOT_LENGTH = json.loads(result.stdout)["scSlotLength"]
SLOT_LENGTH

1000

In [16]:
SLOT_OFFSET = json.loads(result.stdout)["scSlotZeroTime"]
SLOT_OFFSET

1666656000000

**Tip of the Blockchain**

In [17]:
cmd = [cardanoCLI, "query", "tip", 
       "--testnet-magic", MAGIC]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

{
    "block": 300636,
    "epoch": 72,
    "era": "Babbage",
    "hash": "c180cbb9394a28e8f0edc602839a6ce5a85d1d9d206f123d75aa1a9dee37e9d0",
    "slot": 6273783,
    "syncProgress": "100.00"
}




In [18]:
TIP = json.loads(result.stdout)["slot"]
TIP

6273783

In [19]:
NOW = TIP*SLOT_LENGTH+SLOT_OFFSET
NOW

1672929783000

In [20]:
HOUR = 3600*1000

In [21]:
import datetime

def convert_timestamp(timestamp):
    return datetime.datetime.fromtimestamp(timestamp / 1000)

print(convert_timestamp(NOW))
# print(convert_timestamp(NOW))

2023-01-05 15:43:03


**mint tokens**

In [22]:
MINT_EXPIRES = TIP + 1000000
MINT_EXPIRES

7273783

In [25]:
PARTY1_ADDRESS+":"+PARTY1_SKEYFILE

'addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5:wallet1.skey'

In [27]:
PARTY1_ADDRESS

'addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5'

In [28]:
PARTY2_ADDRESS

'addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj'

In [ ]:
 # !!! make sure you send some funds to the addresses !!!
    

In [33]:
cmd = [marloweCLI, "util", "mint", 
       "--testnet-magic", MAGIC,             # '2'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--issuer", PARTY1_ADDRESS+":"+PARTY1_SKEYFILE, # 'addr_test1vqhrmz3u5sawlys6tfctctqv78c5d8k4x6reac0l992m46cd95fee:wallet1.skey'
#        "--required-signer", PARTY1_SKEYFILE,
#        "--change-address", PARTY1_ADDRESS,
       "--count", str(1),
       "--expires", str(MINT_EXPIRES),
       "--out-file", "mint.raw",
       "--submit", str(600),
#        PARTY1_TOKENASSETNAME, PARTY2_TOKENASSETNAME]   # 'Party1', 'Party2', do not use quotes "PARTY1_TOKENASSETNAME"
       PARTY1_TOKENASSETNAME+":"+PARTY1_ADDRESS,
       PARTY2_TOKENASSETNAME+":"+PARTY2_ADDRESS]   # 'Party1', 'Party2', do not use quotes "PARTY1_TOKENASSETNAME"
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

PolicyID "8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8"


Fee: Lovelace 179405
Size: 354 / 16384 = 2%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%



In [34]:
# check wallet1 and wallet2
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,067a19390e6ce36ca39c94cf3cc58291eb226378092608fb795cfc336e08a4d6,0,247751795,NaN,NaN,None,0,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,067a19390e6ce36ca39c94cf3cc58291eb226378092608fb795cfc336e08a4d6,1,1034400,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN
2,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,067a19390e6ce36ca39c94cf3cc58291eb226378092608fb795cfc336e08a4d6,2,1034400,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747932,Party2,1,NaN
3,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,83f3ba623421332a8787149476d07becbc9432dfc3ba1a55607021c503f4f877,0,250000000,NaN,NaN,None,0,NaN


In [35]:
cmd = [marloweCLI, "util", "clean", 
       "--testnet-magic", MAGIC,             # '2'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--required-signer", PARTY1_SKEYFILE,
       "--change-address", PARTY1_ADDRESS,
       "--out-file", "/dev/null",
       "--submit", str(600)]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c"




In [36]:
# check wallet1 and wallet2
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c,0,246608726,NaN,NaN,None,0,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN
2,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,067a19390e6ce36ca39c94cf3cc58291eb226378092608fb795cfc336e08a4d6,2,1034400,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747932,Party2,1,NaN
3,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,83f3ba623421332a8787149476d07becbc9432dfc3ba1a55607021c503f4f877,0,250000000,NaN,NaN,None,0,NaN


In [37]:
cmd = [marloweCLI, "util", "clean", 
       "--testnet-magic", MAGIC,             # '2'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--required-signer", PARTY2_SKEYFILE,
       "--change-address", PARTY2_ADDRESS,
       "--out-file", "/dev/null",
       "--submit", str(600)]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943"




In [38]:
# check wallet1 and wallet2
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c,0,246608726,NaN,NaN,None,0,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN
2,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,0,248856931,NaN,NaN,None,0,NaN
3,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747932,Party2,1,NaN


In [ ]:
# party1 = ffee2ed84fd6eca8628fc4714518f52f68bbbe9be8b8e427199640b935af68c4#0 (68c4#0) will spend UTXO
# as well as ffee2ed84fd6eca8628fc4714518f52f68bbbe9be8b8e427199640b935af68c4#1 (68c4#1) and 
# spend the tokens or use that as proof 150ad48b834239dd9a8e74fb7185a3a4be73ba51fdd3da49f6c8036e.Party1 (036e.Party)

In [ ]:
# the contract requires a minimum ADA requirement and 2x timeouts.
# it specifies that party1 will swap 200 ADA for 100 ADA (no tokens)
# this must happen before two deadlines ! 

In [39]:
MINIMUM_ADA           = 3*ADA           # The minimum lovelace to be included with native token output

In [40]:
PARTY1_TIMEOUT = NOW+12*HOUR

In [41]:
PARTY2_TIMEOUT = NOW+24*HOUR

In [42]:
ADA                   = 1000000         # 1 ADA = 1000000 lovelace
# MILLISECOND           = 1000
# NOW                   = round(time.time())*1000
# HOUR                  = 60*60*MILLISECOND
# PAYMENT_DEADLINE      = NOW+10*HOUR     # The payment deadline, ten hours from now
# MINIMUM_ADA           = 3*ADA           # The minimum lovelace to be included with native token output

PARTY1_TOKENASSETNAME = 'Party1'        # this is 'Buyer', 'wallet 1'
PARTY2_TOKENASSETNAME = 'Party2'        # this is 'Seller', 'wallet 2'
PARTY1_AMOUNT         = 200*ADA         # 200000000, means we will hard-code this swap amount in the script
PARTY2_AMOUNT         = 100*ADA         # 100000000, means we will hard-code this swap amount in the script
PARTY1_ADDRESS        = open('wallet1.address', 'r').read() # string value 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
PARTY2_ADDRESS        = open('wallet2.address', 'r').read() # string value 'addr_test1vrfhx0w0fy9aad9kskezy5znpmey0h335gcjen2pp4gjj2s2ax92x'
PARTY1_SKEYFILE       = 'wallet1.skey'  # string value pointing to path/file
PARTY2_SKEYFILE       = 'wallet2.skey'
PARTY1_VKEYFILE       = 'wallet1.vkey'
PARTY2_VKEYFILE       = 'wallet2.vkey'

# MAGIC                 = '2'          # marlowe-pioneers testnet ID = 1567 (provided by developers)
# SOCKET                = '/tmp/node.socket'

# the values below will be created further down the road, 
# it can be interesting to list them here for your understanding
TOKEN_POLICY_ID       = ''  # minted token Policy ID, '0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43'
TX1_IN                = ''  # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9'
TX1_PARTY1_ADA        = ''  # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9#0'
TX1_PARTY1_TOKENS     = ''  # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9#1'
TX1_PARTY2_ADA        = ''  #
TX1_PARTY2_TOKENS     = ''  #
TX2_IN                = ''  # 'f773c71f9b58dd95fd470aedd9df800838e841aef807102abcd87e3c881fd373'
CONTRACT_ADDRESS      = ''  # 'addr_test1wp544j8t4m3a32g6apad3lpjjfj09j948y8m4nthc5e9rgg5uxu2z'
REDEEM_ADDRESS        = ''  # 'addr_test1wr0970a9k82v6pcw68x2yzwm737zyepg9y8ar8jxsxstf6gzpsmyv'

In [80]:
# experiment generate contract from template
# minted tokenID.tokenNAME == currency_symbol.token_name
PARTY1_TOKENID   = "150ad48b834239dd9a8e74fb7185a3a4be73ba51fdd3da49f6c8036e"
PARTY1_TOKENNAME = "Party1"
PARTY2_TOKENID   = "150ad48b834239dd9a8e74fb7185a3a4be73ba51fdd3da49f6c8036e"
PARTY2_TOKENNAME = "Party2"

# this example seem not to work for swapping ADA, don't understand the syntax (we will not use these files)
cmd = [marloweCLI, "template", "swap", 
       "--minimum-ada", str(MINIMUM_ADA),                # '3000000'
       "--a-party", "Role="+PARTY1_TOKENASSETNAME,       # 'Role=Party1'
       "--a-token", PARTY1_TOKENID+"."+PARTY1_TOKENNAME, # '0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.BERRY'
       "--a-amount", str(PARTY1_AMOUNT),                 # '200000000'
       "--a-timeout", str(PARTY1_TIMEOUT),             # '1658275769000'
       "--b-party", "Role="+PARTY2_TOKENASSETNAME,       # 'Role=Party2'
       "--b-token", PARTY2_TOKENID+"."+PARTY2_TOKENNAME, # '0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.CHOC'
       "--b-amount", str(PARTY2_AMOUNT),                 # '100000000'
       "--b-timeout", str(PARTY2_TIMEOUT),             # '1658275769000'
       "--out-contract-file", "swap_template.contract",
       "--out-state-file", "swap_template.state"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

In [82]:
print(convert_timestamp(1672946434000))
print(convert_timestamp(1672989634000))

2023-01-05 20:20:34
2023-01-06 08:20:34


In [81]:
!cat swap_template.contract

{
    "timeout": 1672946434000,
    "timeout_continuation": "close",
    "when": [
        {
            "case": {
                "deposits": 200000000,
                "into_account": {
                    "role_token": "Role=Party1"
                },
                "of_token": {
                    "currency_symbol": "150ad48b834239dd9a8e74fb7185a3a4be73ba51fdd3da49f6c8036e",
                    "token_name": "Party1"
                },
                "party": {
                    "role_token": "Role=Party1"
                }
            },
            "then": {
                "timeout": 1672989634000,
                "timeout_continuation": "close",
                "when": [
                    {
                        "case": {
                            "deposits": 100000000,
                            "into_account": {
                                "role_token": "Role=Party2"
                            },
                            "of_token": {
                     

### script
The goal here is to hard-code substitute parameters in the json script,  
replacing with real values the token name, amounts, timeout values, and so on...  

In [43]:
# currency_symbol = PolicyID of the minted tokens (ex: "0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43")
PARTY1_TOKENID   = ""  # empty (default to ADA)
PARTY2_TOKENID   = ""  # empty (default to ADA)

# token_name = currency name of the minted tokens (ex: "CHOC")
PARTY1_TOKENNAME = ""  # empty (default to ADA)
PARTY2_TOKENNAME = ""  # empty (default to ADA)

TO BE replaced below later these token names !! then it needs to be tested...  
Maybe we can later add the minting of tokens and try to swap these as an example.  

In [44]:
print(convert_timestamp(PARTY1_TIMEOUT))
print(convert_timestamp(PARTY2_TIMEOUT))

2023-01-06 03:43:03
2023-01-06 15:43:03


In [45]:
result = subprocess.run("""cat <<EOF > tx-1.contract
{
  "when": [
    {
      "then": {
        "when": [
          {
            "then": {
              "token": {
                "token_name": "",
                "currency_symbol": ""
              },
              "to": {
                "party": {
                  "role_token": '"""+PARTY2_TOKENASSETNAME+"""'
                }
              },
              "then": {
                "token": {
                  "token_name": "",
                  "currency_symbol": ""
                },
                "to": {
                  "party": {
                    "role_token": '"""+PARTY1_TOKENASSETNAME+"""'
                  }
                },
                "then": "close",
                "pay": """+str(PARTY2_AMOUNT)+""",
                "from_account": {
                  "role_token": '"""+PARTY2_TOKENASSETNAME+"""'
                }
              },
              "pay": """+str(PARTY1_AMOUNT)+""",
              "from_account": {
                "role_token": '"""+PARTY1_TOKENASSETNAME+"""'
              }
            },
            "case": {
              "party": {
                "role_token": '"""+PARTY2_TOKENASSETNAME+"""'
              },
              "of_token": {
                "token_name": "",
                "currency_symbol": ""
              },
              "into_account": {
                "role_token": '"""+PARTY2_TOKENASSETNAME+"""'
              },
              "deposits": """+str(PARTY2_AMOUNT)+"""
            }
          }
        ],
        "timeout_continuation": "close",
        "timeout": """+str(PARTY2_TIMEOUT)+"""
      },
      "case": {
        "party": {
          "role_token": '"""+PARTY1_TOKENASSETNAME+"""'
        },
        "of_token": {
          "token_name": "",
          "currency_symbol": ""
        },
        "into_account": {
          "role_token": '"""+PARTY1_TOKENASSETNAME+"""'
        },
        "deposits": """+str(PARTY1_AMOUNT)+"""
      }
    }
  ],
  "timeout_continuation": "close",
  "timeout": """+str(PARTY1_TIMEOUT)+"""
}
EOF
""", shell=True)

In [46]:
!cat tx-1.contract

{
  "when": [
    {
      "then": {
        "when": [
          {
            "then": {
              "token": {
                "token_name": "",
                "currency_symbol": ""
              },
              "to": {
                "party": {
                  "role_token": 'Party2'
                }
              },
              "then": {
                "token": {
                  "token_name": "",
                  "currency_symbol": ""
                },
                "to": {
                  "party": {
                    "role_token": 'Party1'
                  }
                },
                "then": "close",
                "pay": 100000000,
                "from_account": {
                  "role_token": 'Party2'
                }
              },
              "pay": 200000000,
              "from_account": {
                "role_token": 'Party1'
              }
            },
            "case": {
              "party": {
                "role_token": 'Par

Question: I don't understand yet why minTime always is 1,  
I believe it has to do with the start of the existence of the blockchain.  
One can believe we need to set the startdate NOW in there.  
Question: why is there no secondary account party 2 in here, what is this state file for ?  
is this to do the initial startup of the contract somehow ?  
Question: Why do we need 2 or 3 ADA here ?  
I believe this is to counter bursting of DoS attacks in contracts initialisations...  

In [47]:
result = subprocess.run("""cat <<EOF > tx-1.state
{
  "accounts": [
    [[{ "role_token": '"""+PARTY1_TOKENASSETNAME+"""'}, { "currency_symbol": "", "token_name": "" }], """+str(MINIMUM_ADA)+"""]
  ],
  "choices": [],
  "boundValues": [],
  "minTime": """+str(1)+"""
}
""", shell=True)

In [48]:
!cat tx-1.state

{
  "accounts": [
    [[{ "role_token": 'Party1'}, { "currency_symbol": "", "token_name": "" }], 3000000]
  ],
  "choices": [],
  "boundValues": [],
  "minTime": 1
}


In [35]:
# !cat tx-1.state

{
  "accounts": [
    [[{ "role_token": 'Party1'}, { "currency_symbol": "", "token_name": "" }], 3000000]
  ],
  "choices": [],
  "boundValues": [],
  "minTime": 1
}


### template (optional)
Using a template, one can construct with marlowe-cli the *.contract and *.state files  
Today these templates are available:  
  * escrow
  * simple
  * swap
  * zcb (zero-coupon bond)
  * coveredCall

In [25]:
# minted tokenID.tokenNAME == currency_symbol.token_name
PARTY1_TOKENID   = "0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43"
PARTY1_TOKENNAME = "BERRY"
PARTY2_TOKENID   = "0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43"
PARTY2_TOKENNAME = "CHOC"

```text
marlowe-cli template swap:

  --minimum-ada INTEGER             Lovelace that the first party contributes to the initial state
  --a-party PARTY                   The first party
  --a-token TOKEN                   The first party's token
  --a-amount INTEGER                The amount of the first party's token
  --a-timeout POSIX_TIME            The timeout for the first party's deposit, in POSIX milliseconds
  --b-party PARTY                   The second party
  --b-token TOKEN                   The second party's token
  --b-amount INTEGER                The amount of the second party's token
  --b-timeout POSIX_TIME            The timeout for the second party's deposit, in POSIX milliseconds
  --out-contract-file CONTRACT_FILE JSON output file for the contract
  --out-state-file STATE_FILE       JSON output file for the contract's state

```

In [26]:
# this example seem not to work for swapping ADA, don't understand the syntax (we will not use these files)
cmd = [marloweCLI, "template", "swap", 
       "--minimum-ada", str(MINIMUM_ADA),                # '3000000'
       "--a-party", "Role="+PARTY1_TOKENASSETNAME,       # 'Role=Party1'
       "--a-token", PARTY1_TOKENID+"."+PARTY1_TOKENNAME, # '0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.BERRY'
       "--a-amount", str(PARTY1_AMOUNT),                 # '200000000'
       "--a-timeout", str(PAYMENT_DEADLINE),             # '1658275769000'
       "--b-party", "Role="+PARTY2_TOKENASSETNAME,       # 'Role=Party2'
       "--b-token", PARTY2_TOKENID+"."+PARTY2_TOKENNAME, # '0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.CHOC'
       "--b-amount", str(PARTY2_AMOUNT),                 # '100000000'
       "--b-timeout", str(PAYMENT_DEADLINE),             # '1658275769000'
       "--out-contract-file", "swap_template.contract",
       "--out-state-file", "swap_template.state"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

In [27]:
!cat swap_template.contract

{
    "timeout": 1672874750000,
    "timeout_continuation": "close",
    "when": [
        {
            "case": {
                "deposits": 200000000,
                "into_account": {
                    "role_token": "Role=Party1"
                },
                "of_token": {
                    "currency_symbol": "0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43",
                    "token_name": "BERRY"
                },
                "party": {
                    "role_token": "Role=Party1"
                }
            },
            "then": {
                "timeout": 1672874750000,
                "timeout_continuation": "close",
                "when": [
                    {
                        "case": {
                            "deposits": 100000000,
                            "into_account": {
                                "role_token": "Role=Party2"
                            },
                            "of_token": {
                      

In [28]:
!cat swap_template.state

{
    "accounts": [
        [
            [
                {
                    "role_token": "Role=Party1"
                },
                {
                    "currency_symbol": "",
                    "token_name": ""
                }
            ],
            3000000
        ]
    ],
    "boundValues": [],
    "choices": [],
    "minTime": 1
}

## Transactions

Extended Unspent Transaction Output (EUTXO) model in the Cardano system represent  
unspent transaction outputs that can be controlled by a specific user's wallet,  
used to track cryptocurrency balances and  
consumed as inputs in new transactions to transfer value on the network.

[EUTxO handbook (*.pdf)](../../docs/pdf/EUTXO_handbook.pdf)

![handbook - "what is a transaction?"](../../image/EUTXO_handbook_transaction.png) 

In [91]:
from IPython.display import IFrame

# visual of the steps the contract will go through
url = "https://miro.com/app/board/uXjVP3tf9_E=/?moveToWidget=3458764542137977013&cot=14"
IFrame(url, width="800", height="600")

## Funding

### faucet
Create a new wallet on the "Preview" network using a light wallet webapp.  
Request to send tADA (test ADA) to your light wallet.  
https://developers.cardano.org/docs/integrate-cardano/testnet-faucet/#testnets-faucet  

Transfer **500 tADA** to wallet1 and wallet2.    

In [64]:
PARTY1_ADDRESS

'addr_test1vr4gmk46x82e49cfr2hmn9xtc3ftwjmtdjg9n79wdx4nx7c3j7av2'

In [65]:
PARTY2_ADDRESS

'addr_test1vzqjr3axkzxu6273arrjcy272cyurzawzd7kd9ncdu995jql3qxe5'

In [94]:
# check wallet1 and wallet2
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vqhrmz3u5sawlys6tfctctqv78c5d8k4x6reac0l992m46cd95fee,ffee2ed84fd6eca8628fc4714518f52f68bbbe9be8b8e427199640b935af68c4,0,246608726,NaN,NaN,None,0,NaN
1,addr_test1vqhrmz3u5sawlys6tfctctqv78c5d8k4x6reac0l992m46cd95fee,ffee2ed84fd6eca8628fc4714518f52f68bbbe9be8b8e427199640b935af68c4,1,2000000,150ad48b834239dd9a8e74fb7185a3a4be73ba51fdd3da49f6c8036e,506172747931,Party1,1,NaN
2,addr_test1vrwycxp7gx8gcf2wgzkkpuptrpe55ejlrcyntfc0yz6j75qjal5w7,032f3af51c9fa3e577aeffcaf3c097109d3cfbfd017da4e5853e2f6a161364fa,0,248856931,NaN,NaN,None,0,NaN
3,addr_test1vrwycxp7gx8gcf2wgzkkpuptrpe55ejlrcyntfc0yz6j75qjal5w7,032f3af51c9fa3e577aeffcaf3c097109d3cfbfd017da4e5853e2f6a161364fa,1,2000000,150ad48b834239dd9a8e74fb7185a3a4be73ba51fdd3da49f6c8036e,506172747932,Party2,1,NaN


In [518]:
# # check wallet1 and wallet2
# pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
2,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
3,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN


## Tokens

### minting
We chose to mint 2 tokens using wallet 1, one for each role.  
We pay a fee for minting these tokens from this wallet.  
One can argue to mint on any wallet and distribute.  

```text
marlowe-cli util mint:

  --testnet-magic INTEGER        Network magic
  --socket-path SOCKET_FILE      Location of the cardano-node socket file
  --required-signer SIGNING_FILE File containing a required signing key
  --metadata-file JSON_FILE      The CIP-25 metadata, with keys for each token name
  --count INTEGER                The number of each token to mint
  --expires SLOT_NO              The slot number after which miniting is no longer possible
  --lovelace LOVELACE            The lovelace to send with each bundle of tokens
  --change-address ADDRESS       Address to receive ADA in excess of fee
  --out-file FILE                Output file for transaction body
  --submit SECONDS               Also submit the transaction, and wait for confirmation
  TOKEN_NAME                     The name of the token
```

In [66]:
marloweCLI

'/home/ubuntu/cardano/marlowe-cli'

In [67]:
MAGIC

'2'

In [68]:
SOCKET

'/tmp/node.socket'

In [69]:
PARTY1_SKEYFILE

'wallet1.skey'

In [70]:
PARTY1_ADDRESS

'addr_test1vr4gmk46x82e49cfr2hmn9xtc3ftwjmtdjg9n79wdx4nx7c3j7av2'

In [71]:
PARTY2_ADDRESS

'addr_test1vzqjr3axkzxu6273arrjcy272cyurzawzd7kd9ncdu995jql3qxe5'

In [72]:
PARTY1_ADDRESS+":"+PARTY1_SKEYFILE

'addr_test1vr4gmk46x82e49cfr2hmn9xtc3ftwjmtdjg9n79wdx4nx7c3j7av2:wallet1.skey'

In [73]:
pwd

'/home/ubuntu/notebooks/cardano-contracts-cookbook/code/notebooks'

In [74]:
PARTY1_TOKENASSETNAME

'Party1'

In [75]:
PARTY2_TOKENASSETNAME

'Party2'

In [43]:
cmd = [marloweCLI, "util", "mint", 
       "--testnet-magic", MAGIC,             # '1567'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--issuer", PARTY1_ADDRESS+":"+PARTY1_SKEYFILE, # 'wallet1.skey'
#        "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--out-file", "mint.raw",
       "--submit", str(600),
       PARTY1_TOKENASSETNAME+":"+PARTY1_ADDRESS,
       PARTY2_TOKENASSETNAME+":"+PARTY2_ADDRESS]   # 'Party1', 'Party2', do not use quotes "PARTY1_TOKENASSETNAME"
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

PolicyID "82335d282571a0381620ee04a969bcb44d45162ca44e08a8cb5134e0"


Fee: Lovelace 178613
Size: 336 / 16384 = 2%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%



In [40]:
# cmd = [marloweCLI, "util", "mint", 
#        "--testnet-magic", MAGIC,             # '1567'
#        "--socket-path", SOCKET,              # '/tmp/node.socket'
#        "--issuer", PARTY1_ADDRESS+":"+PARTY1_SKEYFILE, # 'wallet1.skey'
# #        "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
#        "--out-file", "mint.raw",
#        "--submit", str(600),
#        PARTY1_TOKENASSETNAME+":"+PARTY1_ADDRESS,
#        PARTY2_TOKENASSETNAME+":"+PARTY2_ADDRESS]   # 'Party1', 'Party2', do not use quotes "PARTY1_TOKENASSETNAME"
# result = subprocess.run(cmd, capture_output=True, text=True)
# print(result.stdout)
# print(result.stderr)

PolicyID "bccf65309eebf2c982c2a75fe7a0b61ff5f06c6fb5baacaa9d34804f"


Fee: Lovelace 178613
Size: 336 / 16384 = 2%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%



In [67]:
# cmd = [marloweCLI, "util", "mint", 
#        "--testnet-magic", MAGIC,             # '1567'
#        "--socket-path", SOCKET,              # '/tmp/node.socket'
#        "--issuer", PARTY1_ADDRESS+":"+PARTY1_SKEYFILE, # 'wallet1.skey'
# #        "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
#        "--out-file", "mint.raw",
#        "--submit", str(600),
#        PARTY1_TOKENASSETNAME+":"+PARTY1_ADDRESS,
#        PARTY2_TOKENASSETNAME+":"+PARTY2_ADDRESS]   # 'Party1', 'Party2', do not use quotes "PARTY1_TOKENASSETNAME"
# result = subprocess.run(cmd, capture_output=True, text=True)
# print(result.stdout)
# print(result.stderr)


Expecting token name and recipient address in the following format: TOKENNAME:ADDRESS

Usage: marlowe-cli util mint --testnet-magic INTEGER [--socket-path SOCKET_FILE]
                             --issuer ADDRESS:SIGNING_FILE 
                             [(--token-provider ADDRESS:SIGNING_FILE)] 
                             [--metadata-file JSON_FILE] [--count INTEGER] 
                             [--expires SLOT_NO] --out-file FILE 
                             [--submit SECONDS] TOKEN_NAME:ADDRESS

  Mint native tokens.



In [ ]:
# marlowe-cli util mint --testnet-magic 2 --socket-path /tmp/node.socket --issuer addr_test1vr6a2sahjkcvsjsyycnxwpl6gvedr3kwdpyx9qg5r59zqxc78qws9:/home/ubuntu/notebooks/cardano-contracts-cookbook/code/notebooks/wallet1.skey --out-file mint.raw --submit 600 Party1:addr_test1vr6a2sahjkcvsjsyycnxwpl6gvedr3kwdpyx9qg5r59zqxc78qws9 Party2:addr_test1vz0s4g2hemdggqtfa2qnlnscpq4xhtrnujqjtj4k748v9ts4jj5hd
            
# Fee: Lovelace 178789
# Size: 340 / 16384 = 2%
# Execution units:
#   Memory: 0 / 14000000 = 0%
#   Steps: 0 / 10000000000 = 0%
# PolicyID "6939fd126ef37f54cd0289af07fb029e885598c70a2877cd22e54d06"

In [41]:
# cmd = [marloweCLI, "util", "mint", 
#        "--testnet-magic", MAGIC,             # '1567'
#        "--socket-path", SOCKET,              # '/tmp/node.socket'
#        "--required-signer", PARTY1_SKEYFILE, # 'wallet1.skey'
#        "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
#        "--out-file", "mint.raw",
#        "--submit", str(600),
#        PARTY1_TOKENASSETNAME, PARTY2_TOKENASSETNAME]   # 'Party1', 'Party2', do not use quotes "PARTY1_TOKENASSETNAME"
# result = subprocess.run(cmd, capture_output=True, text=True)
# print(result.stdout)
# print(result.stderr)

PolicyID "6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508"




In [73]:
# TOKEN_POLICY_ID = "6939fd126ef37f54cd0289af07fb029e885598c70a2877cd22e54d06"

In [44]:
# The policy ID will be used for the Marlowe roles currency.  
# Save this minted token Policy ID variable.  
TOKEN_POLICY_ID = result.stdout.split('"')[1]  
TOKEN_POLICY_ID

'82335d282571a0381620ee04a969bcb44d45162ca44e08a8cb5134e0'

In [45]:
# check wallet1 and wallet2
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpec6ur80es2rkkr7j9vg2ge7zjjjnnrdhdnctts66mmw0s6tfk7e,72ca915acae588a624c0b8705de00e1d85d1438260443976f0d9c33a8ad5c840,0,1047752587,NaN,NaN,None,0,NaN
1,addr_test1vpec6ur80es2rkkr7j9vg2ge7zjjjnnrdhdnctts66mmw0s6tfk7e,72ca915acae588a624c0b8705de00e1d85d1438260443976f0d9c33a8ad5c840,1,1034400,82335d282571a0381620ee04a969bcb44d45162ca44e08a8cb5134e0,506172747931,Party1,1,NaN
2,addr_test1vpec6ur80es2rkkr7j9vg2ge7zjjjnnrdhdnctts66mmw0s6tfk7e,72ca915acae588a624c0b8705de00e1d85d1438260443976f0d9c33a8ad5c840,2,1034400,82335d282571a0381620ee04a969bcb44d45162ca44e08a8cb5134e0,506172747932,Party2,1,NaN
3,addr_test1vpka6uspkmw89l68vrq8l62d4yk5a43xx4ryrfqyxqdqcyczt8y0e,NaN,0,0,NaN,NaN,NaN,0,NaN


In [519]:
# # check wallet1 and wallet2
# pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
2,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
3,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN


In [46]:
TX0_IN = "72ca915acae588a624c0b8705de00e1d85d1438260443976f0d9c33a8ad5c840"

In [44]:
# TX0_IN = "e595b9fc55e0df7f8e0b8522628b8150e37cf249b4265e651e089cbf150ef3ea"

### distribute
send the two tokens created with wallet 1 out to each participant in one transaction.  
wallet 1 must receive 'Party1' token and some minimum currency (3 tADA),  
wallet 2 must receive 'Party2' token and some minimum currency (3 tADA).  

Alternatively, one can distribute the tokens using any wallet software like Daedalus as well.

```text
marlowe-cli transaction simple:

  --testnet-magic INTEGER         Network magic
  --socket-path SOCKET_FILE       Location of the cardano-node socket file
  --required-signer SIGNING_FILE  File containing a required signing key
  --tx-in TXID#TXIX               Transaction input in TxId#TxIx format
  --tx-out ADDRESS+VALUE          Transaction output in ADDRESS+VALUE format
  --change-address                ADDRESS Address to receive ADA in excess of fee
  --metadata-file METADATA_FILE   JSON file containing metadata
  --out-file FILE                 Output file for transaction body
  --submit SECONDS                Also submit the transaction, and wait for confirmation
  --print-stats                   Print statistics
  --script-invalid                Assert that the transaction is invalid
```

In [47]:
MINIMUM_ADA

3000000

In [48]:
TX0_IN

'72ca915acae588a624c0b8705de00e1d85d1438260443976f0d9c33a8ad5c840'

In [49]:
PARTY1_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY1_TOKENASSETNAME

'addr_test1vpec6ur80es2rkkr7j9vg2ge7zjjjnnrdhdnctts66mmw0s6tfk7e+3000000+1 82335d282571a0381620ee04a969bcb44d45162ca44e08a8cb5134e0.Party1'

In [50]:
PARTY2_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY2_TOKENASSETNAME

'addr_test1vpka6uspkmw89l68vrq8l62d4yk5a43xx4ryrfqyxqdqcyczt8y0e+3000000+1 82335d282571a0381620ee04a969bcb44d45162ca44e08a8cb5134e0.Party2'

In [ ]:
hier ontbreekt in feite een wallet2.skey --required-signer moet er mogelijks nog eens staan...  
dit komt omdat de 2e token hier uitzonderlijk al op het 2e adres staat !

marlowe-cli transaction simple --help
Usage: marlowe-cli util mint --testnet-magic INTEGER [--socket-path SOCKET_FILE]  
                             --issuer ADDRESS:SIGNING_FILE  
                             [(--token-provider ADDRESS:SIGNING_FILE)]  
                             [--metadata-file JSON_FILE] [--count INTEGER]  
                             [--expires SLOT_NO] --out-file FILE  
                             [--submit SECONDS] TOKEN_NAME:ADDRESS  

bij --testnet-magic is het verplicht
bij [--socket-path SOCKET_FILE] rechte [ ] bedoelen ze optioneel maar max 1 
bij (--required-signer SIGNING_FILE) met ( ) bedoelen ze optioneel maar min 1 of meer

ik heb het hieronder aangepast maar nog niet getest, misschien beter bovenaan de tokens nog niet verdelen ?
we hebben ze bovenaan al wel verdeeld... maar er is dus nog te weinig ADA in elke wallet (opgelet in lovelace !)
misschien moeten we eerst de faucet doen en dan een transfer, ik vind deze complexe methode om in 1 keer ADA en tokens te verdelen wel cool !

In [51]:
# distribute the tokens that were minted on wallet1 around all wallets.

cmd = [marloweCLI, "transaction", "simple",  # Build a non-Marlowe transaction
       "--testnet-magic", MAGIC,             # '1567'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--required-signer", PARTY1_SKEYFILE, # 'wallet1.skey'
       "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--tx-in", TX0_IN+"#0",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#0'
       "--tx-in", TX0_IN+"#1",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#1'
       "--tx-in", TX0_IN+"#2",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#2'
       "--tx-out", PARTY1_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY1_TOKENASSETNAME,
       # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party1'
       "--tx-out", PARTY2_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY2_TOKENASSETNAME,
       # 'addr_test1vrfhx0w0fy9aad9kskezy5znpmey0h335gcjen2pp4gjj2s2ax92x+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party2'
       "--out-file", "/dev/null",            # '/dev/null'
       "--submit", str(600)]                 # '600'
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "59ee1d14470afa51d6d7d3aebb29f0dfde74e4458a7704518189755d82a8ea49"




In [90]:
# distribute the tokens that were minted on wallet1 around all wallets.

cmd = [marloweCLI, "transaction", "simple",  # Build a non-Marlowe transaction
       "--testnet-magic", MAGIC,             # '1567'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--required-signer", PARTY1_SKEYFILE, # 'wallet1.skey'
       "--required-signer", PARTY2_SKEYFILE, # 'wallet1.skey'
       "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--tx-in", TX0_IN+"#0",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#0'
       "--tx-in", TX0_IN+"#1",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#1'
       "--tx-in", TX0_IN+"#2",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#2'
       "--tx-out", PARTY1_ADDRESS+"+"+str(500*ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY1_TOKENASSETNAME,
       # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party1'
       "--tx-out", PARTY2_ADDRESS+"+"+str(500*ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY2_TOKENASSETNAME,
       # 'addr_test1vrfhx0w0fy9aad9kskezy5znpmey0h335gcjen2pp4gjj2s2ax92x+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party2'
       "--out-file", "/dev/null",            # '/dev/null'
       "--submit", str(600)]                 # '600'
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)


FileError "wallet1.vkey" (TextEnvelopeTypeError [TextEnvelopeType "PaymentSigningKeyShelley_ed25519",TextEnvelopeType "PaymentExtendedSigningKeyShelley_ed25519_bip32"] (TextEnvelopeType "PaymentExtendedVerificationKeyShelley_ed25519_bip32"))



In [81]:
PARTY2_ADDRESS+"+"+str(500)

'addr_test1vz0s4g2hemdggqtfa2qnlnscpq4xhtrnujqjtj4k748v9ts4jj5hd+500'

In [91]:
# distribute the tokens that were minted on wallet1 around all wallets.

cmd = [marloweCLI, "transaction", "simple",  # Build a non-Marlowe transaction
       "--testnet-magic", MAGIC,             # '1567'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--required-signer", PARTY1_SKEYFILE, # 'wallet1.skey'
       "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--tx-in", TX0_IN+"#0",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#0'
#        "--tx-in", TX0_IN+"#1",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#1'
#        "--tx-in", TX0_IN+"#2",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#2'
#        "--tx-out", PARTY1_ADDRESS+"+"+str(500),
       # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party1'
       "--tx-out", PARTY2_ADDRESS+"+"+str(500*ADA),
       # 'addr_test1vrfhx0w0fy9aad9kskezy5znpmey0h335gcjen2pp4gjj2s2ax92x+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party2'
       "--out-file", "/dev/null",            # '/dev/null'
       "--submit", str(600)]                 # '600'
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "4d205e1eed012bb8fd68544e0f5b179d9b50459f66ddd8aaa4fb5a9b39e6d217"




In [337]:
# # distribute the tokens that were minted on wallet1 around all wallets.

# cmd = [marloweCLI, "transaction", "simple",  # Build a non-Marlowe transaction
#        "--testnet-magic", MAGIC,             # '1567'
#        "--socket-path", SOCKET,              # '/tmp/node.socket'
#        "--required-signer", PARTY1_SKEYFILE, # 'wallet1.skey'
#        "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
#        "--tx-in", TX0_IN+"#0",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#0'
#        "--tx-in", TX0_IN+"#1",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#1'
#        "--tx-in", TX0_IN+"#2",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#2'
#        "--tx-out", PARTY1_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY1_TOKENASSETNAME,
#        # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party1'
#        "--tx-out", PARTY2_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY2_TOKENASSETNAME,
#        # 'addr_test1vrfhx0w0fy9aad9kskezy5znpmey0h335gcjen2pp4gjj2s2ax92x+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party2'
#        "--out-file", "/dev/null",            # '/dev/null'
#        "--submit", str(600)]                 # '600'
# result = subprocess.run(cmd, capture_output=True, text=True)
# print(result.stdout)
# print(result.stderr)

TxId "4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17"




In [52]:
# save the transaction id
TX1_IN = result.stdout.split('"')[1]
TX1_IN

'59ee1d14470afa51d6d7d3aebb29f0dfde74e4458a7704518189755d82a8ea49'

In [338]:
# # save the transaction id
# TX1_IN = result.stdout.split('"')[1]
# TX1_IN

'4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17'

#### Check that the funds are distributed correctly
Notice that both wallets now have a common TxHash value.  

In [49]:
# check wallet1 and wallet2
df = pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c,0,246608726,NaN,NaN,None,0,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN
2,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,0,248856931,NaN,NaN,None,0,NaN
3,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747932,Party2,1,NaN


In [520]:
# # check wallet1 and wallet2
# pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
2,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
3,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN


In [61]:
TX1_PARTY1_ADA = df.iloc[0]['TxHash'] + '#0'
# TX1_PARTY1_ADA = df[df['TokenNameAscii'].isnull()]['TxHash'].values[0] + '#0'

In [62]:
# TX1_PARTY1_TOKENS = df[df['TokenNameAscii'] == 'Party1']['TxHash'].values[0] + '#1'
TX1_PARTY1_TOKENS = df.iloc[1]['TxHash'] + '#1'

In [68]:
TX1_PARTY2_ADA = df.iloc[3]['TxHash'] + '#0'

In [64]:
# TX1_PARTY2_TOKENS = df[df['TokenNameAscii'] == 'Party2']['TxHash'].values[0] + '#2'
TX1_PARTY2_TOKENS = df.iloc[2]['TxHash'] + '#2'

In [70]:
print(TX1_PARTY1_ADA)
print(TX1_PARTY1_TOKENS)
print(TX1_PARTY2_ADA)
print(TX1_PARTY2_TOKENS)

59ee1d14470afa51d6d7d3aebb29f0dfde74e4458a7704518189755d82a8ea49#0
59ee1d14470afa51d6d7d3aebb29f0dfde74e4458a7704518189755d82a8ea49#1
94ace4ba222050903354a9a73fca6a80dda49a2f8cdd81f61e8e3e44df8b6c5a#0
59ee1d14470afa51d6d7d3aebb29f0dfde74e4458a7704518189755d82a8ea49#2


In [340]:
# TX1_PARTY1_ADA    = TX1_IN+"#0"
# TX1_PARTY1_TOKENS = TX1_IN+"#1"
# TX1_PARTY2_ADA    = "ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33"+"#3"
# TX1_PARTY2_TOKENS = TX1_IN+"#2"

In [96]:
#let's get the correct info for the initialise below !!

In [51]:
# check wallet1 and wallet2
df = pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c,0,246608726,NaN,NaN,None,0,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN
2,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,0,248856931,NaN,NaN,None,0,NaN
3,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747932,Party2,1,NaN


In [52]:
TX1_PARTY1_ADA   = df.iloc[0]['TxHash'] + '#0'
TX1_PARTY1_TOKEN = df.iloc[1]['TxHash'] + '#1'
TX1_PARTY2_ADA   = df.iloc[2]['TxHash'] + '#0'
TX1_PARTY2_TOKEN = df.iloc[3]['TxHash'] + '#1'
print(TX1_PARTY1_ADA)
print(TX1_PARTY1_TOKEN)
print(TX1_PARTY2_ADA)
print(TX1_PARTY2_TOKEN)

d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c#0
d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c#1
b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943#0
b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943#1


In [53]:
TOKEN_POLICY_ID = df.iloc[1]['TokenPolicyId']
TOKEN_POLICY_ID

'8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8'

In [81]:
# will the validator address be different  ?
# addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv ?

# I don't know why but for some reason this stay the same address
# addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv

## Deployment

### initialize (step1)
Generate the *.marlowe file with json files *.contract, *.state, Plutus data and network info.  
Question: (not sure here) The wallet addresses of the smart contract are not yet created on the blockchain, but known now.  

```text
Initialize the first transaction of a Marlowe contract and write output to a JSON file *.marlowe

marlowe-cli run initialize:

  --testnet-magic INTEGER          Network magic
  --socket-path SOCKET_FILE        Location of the cardano-node socket file
  --stake-address ADDRESS          Stake address
  --roles-currency CURRENCY_SYMBOL The currency symbol for roles
  --contract-file CONTRACT_FILE    JSON input file for the contract
  --state-file STATE_FILE          JSON input file for the contract state
  --out-file OUTPUT_FILE           JSON output file for initialize
  --print-stats                    Print statistics
```

In [54]:
cmd = [marloweCLI, "run", "initialize",     # Initialize the first transaction of a Marlowe contract
       "--testnet-magic", MAGIC,            # '1567'
       "--socket-path", SOCKET,             # '/tmp/node.socket'
       "--roles-currency", TOKEN_POLICY_ID, # '150ad48b834239dd9a8e74fb7185a3a4be73ba51fdd3da49f6c8036e'
       "--contract-file", "tx-1.contract",  # 'tx-1.contract'
       "--state-file", "tx-1.state",        # 'tx-1.state'
       "--out-file", "tx-1.marlowe",        # 'tx-1.marlowe'
       "--print-stats"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)



Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}



In [55]:
!date
!ls -l tx-1*

Thu Jan  5 15:50:50 CET 2023
-rw-rw-r-- 1 ubuntu ubuntu  1738 Jan  5 15:48 tx-1.contract
-rw-rw-r-- 1 ubuntu ubuntu 35545 Jan  5 15:50 tx-1.marlowe
-rw-rw-r-- 1 ubuntu ubuntu  1128 Jan  5 15:15 tx-1.raw
-rw-rw-r-- 1 ubuntu ubuntu   166 Jan  5 15:48 tx-1.state


In [56]:
# show content of file 
!cat tx-1.marlowe

{
    "era": "babbage",
    "plutusVersion": "PlutusScriptV2",
    "tx": {
        "continuations": [],
        "contract": {
            "timeout": 1672972983000,
            "timeout_continuation": "close",
            "when": [
                {
                    "case": {
                        "deposits": 200000000,
                        "into_account": {
                            "role_token": "Party1"
                        },
                        "of_token": {
                            "currency_symbol": "",
                            "token_name": ""
                        },
                        "party": {
                            "role_token": "Party1"
                        }
                    },
                    "then": {
                        "timeout": 1673016183000,
                        "timeout_continuation": "close",
                        "when": [
                            {
                                "case": {
               

In [345]:
# # show content of file 
# !cat tx-1.marlowe

{
    "state": {
        "choices": [],
        "boundValues": [],
        "accounts": [
            [
                [
                    {
                        "role_token": "Party1"
                    },
                    {
                        "currency_symbol": "",
                        "token_name": ""
                    }
                ],
                3000000
            ]
        ],
        "minTime": 1
    },
    "payments": [],
    "contract": {
        "timeout": 1658454012000,
        "when": [
            {
                "then": {
                    "timeout": 1658454012000,
                    "when": [
                        {
                            "then": {
                                "then": {
                                    "then": "close",
                                    "to": {
                                        "party": {
                                            "role_token": "Party1"
                                

### initial deposit (step2)
In the state file it was hard-coded to pay the initial deposit of 3 ADA,  
in order for the contract to be activated/initialized.  
A transaction fee will also be taken.  

```text
Run a Marlowe transaction

marlowe-cli run execute:

  --testnet-magic INTEGER          Network magic
  --socket-path SOCKET_FILE        Location of the cardano-node socket file
  --marlowe-in-file MARLOWE_FILE   JSON file with the Marlowe initial state and initial contract
  --tx-in-marlowe TXID#TXIX        UTxO spent from Marlowe contract
  --tx-in-collateral TXID#TXIX     Collateral for transaction
  --marlowe-out-file MARLOWE_FILE  JSON file with the Marlowe inputs, final state, and final contract
  --tx-in TXID#TXIX                Transaction input in TxId#TxIx format
  --tx-out ADDRESS+VALUE           Transaction output in ADDRESS+VALUE format
  --change-address ADDRESS         Address to receive ADA in excess of fee
  --required-signer SIGNING_FILE   File containing a required signing key
  --metadata-file METADATA_FILE    JSON file containing metadata
  --out-file FILE                  Output file for transaction body
  --submit SECONDS                 Also submit the transaction, and wait for confirmation
  --print-stats                    Print statistics
  --script-invalid                 Assert that the transaction is invalid
```

In [57]:
TX1_PARTY1_ADA

'd007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c#0'

In [58]:
PARTY1_ADDRESS

'addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5'

In [87]:
# this should automatically 'stated in the contract' send the minimum 3 ADA + Fee to the contract. 
# this is not specified in the CLI command to send 3 ADA, but the tx-1.marlowe does...
# --marlowe-out-file
# JSON file with the Marlowe inputs, final state, and final contract
# the "out" do not mean it is an output file created, instead it reads in the script.
# this will spend 3.191549 tADA and generate another transaction to the contract address !

In [59]:
### Run a marlowe contract on the blockchain 'run execute'
cmd = [marloweCLI, "run", "execute",          # Run a Marlowe transaction
       "--testnet-magic", MAGIC,              # '1567'
       "--socket-path", SOCKET,               # '/tmp/node.socket'
       "--tx-in", TX1_PARTY1_ADA,             # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9#0'
       "--change-address", PARTY1_ADDRESS,    # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--required-signer", PARTY1_SKEYFILE,  # 'wallet1.skey'
       "--marlowe-out-file", "tx-1.marlowe",  # 'tx-1.marlowe' JSON file with the Marlowe inputs, final state, and final contract
       "--out-file", "tx-1.raw",              # 'tx-1.raw'
       "--print-stats",
       "--submit", str(600)]                  # '600'
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "4dd84cf29e802861f44b64d9cfe503b49e008add8ad3923edf395d4d0459eba1"


Fee: Lovelace 191549
Size: 527 / 16384 = 3%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%



In [346]:
# # Run a marlowe contract on the blockchain 'run execute'
# cmd = [marloweCLI, "run", "execute",          # Run a Marlowe transaction
#        "--testnet-magic", MAGIC,              # '1567'
#        "--socket-path", SOCKET,               # '/tmp/node.socket'
#        "--tx-in", TX1_PARTY1_ADA,             # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9#0'
#        "--change-address", PARTY1_ADDRESS,    # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
#        "--required-signer", PARTY1_SKEYFILE,  # 'wallet1.skey'
#        "--marlowe-out-file", "tx-1.marlowe",  # 'tx-1.marlowe' JSON file with the Marlowe inputs, final state, and final contract
#        "--out-file", "tx-1.raw",              # 'tx-1.raw'
#        "--print-stats",
#        "--submit", str(600)]                  # '600'
# result = subprocess.run(cmd, capture_output=True, text=True)
# print(result.stdout)
# print(result.stderr)

TxId "7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb"


Fee: Lovelace 187545
Size: 488 / 32768 = 1%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%



In [60]:
# save the transaction id
TX2_IN = result.stdout.split('"')[1]
TX2_IN

'4dd84cf29e802861f44b64d9cfe503b49e008add8ad3923edf395d4d0459eba1'

In [347]:
# # save the transaction id
# TX2_IN = result.stdout.split('"')[1]
# TX2_IN

'7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb'

### prepare contract (stage1-2) 

```text
Prepare the next step of a Marlowe contract and write the output to a JSON file

marlowe-cli run prepare:

  --marlowe-file MARLOWE_FILE     JSON input file for the Marlowe state and contract
  --deposit-account PARTY         The account for the deposit
  --deposit-party PARTY           The party making the deposit
  --deposit-token TOKEN           The token being deposited, if not ADA
  --deposit-amount INTEGER        The amount of token being deposited
  --choice-name NAME              The name of the choice made
  --choice-party PARTY            The party making the choice
  --choice-number INTEGER         The number chosen
  --notify                        Notify the contract
  --invalid-before POSIX_TIME     Minimum time for the input, in POSIX milliseconds
  --invalid-hereafter POSIX_TIME  Maximum time for the input, in POSIX milliseconds
  --out-file OUTPUT_FILE          JSON output file for contract
  --print-stats                   Print statistics
```

In [90]:
# I wonder about the NOW+9*HOUR is nowhere written in the contract itself,
# we need to further make a drawing explaining and testing what the effects and menings are of these deadlines and why we need them...
# question: why is it not str(NOW+10*HOUR) ?
# question: this is the hardcoding of the next stage script, right ?
# question: is this invalid hereafter time something new, for let's say within this time the next wallet must respond or the contract ends ?

In [61]:
"Role="+PARTY1_TOKENASSETNAME

'Role=Party1'

In [62]:
str(PARTY1_AMOUNT)

'200000000'

POSIX time, which is a standard used to represent time as a single numerical value.  
It represents the number of seconds that have elapsed since 00:00:00 UTC on January 1, 1970.  

In [63]:
str(NOW)

'1672929783000'

In [64]:
import datetime

def convert_timestamp(timestamp):
    return datetime.datetime.fromtimestamp(timestamp / 1000)

print(convert_timestamp(NOW))

2023-01-05 15:43:03


In [65]:
str(NOW+10*HOUR)

'1672965783000'

In [66]:
import datetime

def convert_timestamp(timestamp):
    return datetime.datetime.fromtimestamp(timestamp / 1000)

print(convert_timestamp(NOW+10*HOUR))

2023-01-06 01:43:03


In [67]:
now = datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))

2023-01-05 15:51:46


In [68]:
# Minimum time for the input, in POSIX milliseconds.
print("--invalid-before: ", convert_timestamp(NOW))
NOW

--invalid-before:  2023-01-05 15:43:03


1672929783000

In [69]:
#  Maximum time for the input, in POSIX milliseconds.
print("--invalid-hereafter: ", convert_timestamp(NOW+4*HOUR))
NOW+4*HOUR

--invalid-hereafter:  2023-01-05 19:43:03


1672944183000

In [ ]:
# I changed this into this when using v0.0.8.1 and that works somehow
#        "--deposit-account", "Role="+PARTY1_TOKENASSETNAME, # 'Role=Party1'
#        "--deposit-party", "Role="+PARTY1_TOKENASSETNAME,   # 'Role=Party1'

#        "--deposit-account", PARTY1_TOKENASSETNAME, # 'Role=Party1'
#        "--deposit-party", PARTY1_TOKENASSETNAME,   # 'Role=Party1'

In [72]:
# Simulate the operation of a contract
cmd = [marloweCLI, "run", "prepare",                       # Prepare the next step
       "--marlowe-file", "tx-1.marlowe",                   # 'tx-1.marlowe'
       "--deposit-account", PARTY1_TOKENASSETNAME, # 'Role=Party1'
       "--deposit-party", PARTY1_TOKENASSETNAME,   # 'Role=Party1'
#        "--deposit-account", "Role="+PARTY1_TOKENASSETNAME, # 'Role=Party1'
#        "--deposit-party", "Role="+PARTY1_TOKENASSETNAME,   # 'Role=Party1'
       "--deposit-amount", str(PARTY1_AMOUNT),             # '200000000'
       "--invalid-before", str(NOW),                       # '1672903234000' 2023-01-05 08:20:34
       "--invalid-hereafter", str((NOW+4*HOUR)-1),             # '1672917634000' 2023-01-05 12:20:34
       "--out-file", "tx-2.marlowe",                       # 'tx-2.marlowe'
       "--print-stats"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1672929783000},POSIXTime {getPOSIXTime = 1672944182999}), txInputs = [NormalInput (IDeposit "Party1" "Party1" (Token "" "") 200000000)]}


Datum size: 238



In [104]:
# # Simulate the operation of a contract
# cmd = [marloweCLI, "run", "prepare",                       # Prepare the next step
#        "--marlowe-file", "tx-1.marlowe",                   # 'tx-1.marlowe'
#        "--deposit-account", PARTY1_TOKENASSETNAME, # 'Role=Party1'
#        "--deposit-party", PARTY1_TOKENASSETNAME,   # 'Role=Party1'
# #        "--deposit-account", "Role="+PARTY1_TOKENASSETNAME, # 'Role=Party1'
# #        "--deposit-party", "Role="+PARTY1_TOKENASSETNAME,   # 'Role=Party1'
#        "--deposit-amount", str(PARTY1_AMOUNT),             # '200000000'
#        "--invalid-before", str(NOW),                       # '1672903234000' 2023-01-05 08:20:34
#        "--invalid-hereafter", str(NOW+4*HOUR),             # '1672917634000' 2023-01-05 12:20:34
#        "--out-file", "tx-2.marlowe",                       # 'tx-2.marlowe'
#        "--print-stats"]
# result = subprocess.run(cmd, capture_output=True, text=True)
# print(result.stdout)
# print(result.stderr)



Datum size: 238



In [353]:
# # Simulate the operation of a contract
# cmd = [marloweCLI, "run", "prepare",                       # Prepare the next step
#        "--marlowe-file", "tx-1.marlowe",                   # 'tx-1.marlowe'
#        "--deposit-account", "Role="+PARTY1_TOKENASSETNAME, # 'Role=Party1'
#        "--deposit-party", "Role="+PARTY1_TOKENASSETNAME,   # 'Role=Party1'
#        "--deposit-amount", str(PARTY1_AMOUNT),             # '200000000'
#        "--invalid-before", str(NOW),                       # '1658239769000'
#        "--invalid-hereafter", str(NOW+9*HOUR),             # '1658272169000'
#        "--out-file", "tx-2.marlowe",                       # 'tx-2.marlowe'
#        "--print-stats"]
# result = subprocess.run(cmd, capture_output=True, text=True)
# print(result.stdout)
# print(result.stderr)



Datum size: 202



### deposit 200 ADA (step3)

In [73]:
# get the contract address from the smart contract script file
marloweValidatorAddress = !jq -r '.tx.marloweValidator.address' tx-1.marlowe

# save the address
CONTRACT_ADDRESS = marloweValidatorAddress[0]

# weblink to the explorer
print("https://preview.cardanoscan.io/address/"+str(CONTRACT_ADDRESS))

# automatically open the weblink
display(Javascript('window.open("{url}");'.format(url="https://preview.cardanoscan.io/address/"+str(marloweValidatorAddress[0]))))

https://preview.cardanoscan.io/address/addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv


<IPython.core.display.Javascript object>

In [74]:
# get the redeem address from the smart contract script file
marloweValidatorAddress = !jq -r '.tx.rolesValidator.address' tx-1.marlowe

# save the address
REDEEM_ADDRESS = marloweValidatorAddress[0]

# weblink to the explorer
print("https://preview.cardanoscan.io/address/"+str(REDEEM_ADDRESS))

https://preview.cardanoscan.io/address/addr_test1wpyswm4tyqjrmj2xy5glhx9fe7m3n7rwj6fz3qfekly3muckehcu9


In [75]:
# weblink to the explorer for wallet 1
print("https://preview.cardanoscan.io/address/"+str(PARTY1_ADDRESS))

https://preview.cardanoscan.io/address/addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5


In [76]:
# weblink to the explorer for wallet 2
print("https://preview.cardanoscan.io/address/"+str(PARTY2_ADDRESS))

https://preview.cardanoscan.io/address/addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj


In [103]:
# check wallet1, wallet2 and contract, redeem address
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash,210375bd6fd5d03cde2707b2e87114837d45f7ca7ee80278f945f38e,476c6f6265
0,addr_test1vpec6ur80es2rkkr7j9vg2ge7zjjjnnrdhdnctts66mmw0s6tfk7e,59ee1d14470afa51d6d7d3aebb29f0dfde74e4458a7704518189755d82a8ea49,1,3000000,82335d282571a0381620ee04a969bcb44d45162ca44e08a8cb5134e0,506172747931,Party1,1,NaN,NaN,NaN
1,addr_test1vpec6ur80es2rkkr7j9vg2ge7zjjjnnrdhdnctts66mmw0s6tfk7e,5a5241e5547810c7312bf187132d5de3826c8224f0988e4df95c9253f74285cb,0,1040442821,NaN,NaN,None,0,NaN,NaN,NaN
2,addr_test1vpka6uspkmw89l68vrq8l62d4yk5a43xx4ryrfqyxqdqcyczt8y0e,59ee1d14470afa51d6d7d3aebb29f0dfde74e4458a7704518189755d82a8ea49,2,3000000,82335d282571a0381620ee04a969bcb44d45162ca44e08a8cb5134e0,506172747932,Party2,1,NaN,NaN,NaN
3,addr_test1vpka6uspkmw89l68vrq8l62d4yk5a43xx4ryrfqyxqdqcyczt8y0e,94ace4ba222050903354a9a73fca6a80dda49a2f8cdd81f61e8e3e44df8b6c5a,0,350000000,NaN,NaN,None,0,NaN,NaN,NaN
4,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,00cbd222c84bbff42b1c49f718ae21bda1b7ce0488354f51bc6ba815bfa9d2a4,1,1500000,NaN,NaN,NaN,0,3e1b58fb13d4abbf5ee28bc3b911fe015c4fd17dbdcaf0e9eb8da328c7058961,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
139,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,f79c8195c449d6db92e9c3bb517918bf261e69208c9daee27f0ac3cd5d86b078,1,2638045,NaN,NaN,NaN,0,dcfdbc92f23847d6e243e23095883ba99819f6709ce5e579b34abfab2787135a,NaN,NaN
140,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,fa24b9526e85b92f0206b80f2844879704d21f2922789776da05156a43edf577,1,3267446,NaN,NaN,NaN,0,423e5dacd368e19d46eb1b6a8d0e55c341f57bd4a7b1d79d28f6305c80bda581,NaN,NaN
141,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,fce71f23bbf3e51ff7189558a27aca64ef99b16a3b5087ae02b6d1f16cc8589b,1,1500000,NaN,NaN,NaN,0,53e67e13b129d29d8e39a2e4cdf7411de0ebe25521c04dec561a3d67daefd743,NaN,NaN
142,addr_test1wpyswm4tyqjrmj2xy5glhx9fe7m3n7rwj6fz3qfekly3muckehcu9,3ea83fd7d9ecf8b99763de66e00a9dd47e06d5fbeff308bbba4f9fa8d0ebb264,2,1193870,NaN,NaN,NaN,0,NaN,{'5377616e': 500},NaN


In [530]:
# # check wallet1, wallet2 and contract, redeem address
# pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
2,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
3,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN
4,addr_test1wqzctw6fur749cqp8ae9y8e6l6g9uvs9t4pv7thp0zy38cssc9g3z,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,1,3000000,NaN,NaN,NaN,0,0e810f91611be5f0023233e5aa189dd099ad96d05437ffe0bc59a0f9a0137d64
5,addr_test1wqzuqf8hvl9d47d6kftykktrgszgu3yznrrpdgha9naxcmgr8le36,NaN,0,0,NaN,NaN,NaN,0,NaN


In [78]:
# check wallet1, wallet2 and contract, redeem address
df2 = pd.concat([query_df(CONTRACT_ADDRESS)]).reset_index(drop=True)
df2

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,00cbd222c84bbff42b1c49f718ae21bda1b7ce0488354f51bc6ba815bfa9d2a4,1,1500000,NaN,NaN,NaN,0,3e1b58fb13d4abbf5ee28bc3b911fe015c4fd17dbdcaf0e9eb8da328c7058961
1,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,085fab3b6dad7d9ff7fadcdd213a222d97753e84e8a790c04ef3a2b2e2359947,1,1500000,NaN,NaN,NaN,0,3b40695cf4934b693df4a577bdf44c09ce695fa820bd7220b3aff84225b097ab
2,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,08c9c42b9cd4f88024750206afa3e82f87222acc0a801c0d2839aedcc437aa25,1,1500000,NaN,NaN,NaN,0,95f1f75e3ea7e024c1a8964ff1685f5edc18575faab1c15258f9474f51f637dd
3,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,0dbbf357f1231ba34c8ca6ddec4a949e39c803ff17b391c5f72296f2fce1551d,1,1500000,NaN,NaN,NaN,0,6846f46570a5015f50fc43239c9fa13b99af4409253a5428a58bfb4774f14574
4,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,1024a90feccbe2607bdaa0b6a07ee6b189dd563bf078552b7b4790a420235f76,1,1500000,NaN,NaN,NaN,0,aa3b8bbd9ffb8c9688779e46e125c32bdae7601029289067c742a8eca8c5de1c
...,...,...,...,...,...,...,...,...,...
136,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,f4e9580f9900288baaa4a28b2dc429a71fb20bdd1e24ca6ee4b34c98cc37eec2,1,28000000,NaN,NaN,NaN,0,75b0bd86c9df7e5b692bffca2f20b93f5b10b13114d90c5fe88b3b141ba36de9
137,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,f780459264cb4d8c7fb4953334753e6eee38f7388bda4ce1cbcc36d7a4b3ea8f,1,1500000,NaN,NaN,NaN,0,9fa9a26513a6eab57d82c37106fc16eb944d759370157b1e661468f25956c3e5
138,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,f79c8195c449d6db92e9c3bb517918bf261e69208c9daee27f0ac3cd5d86b078,1,2638045,NaN,NaN,NaN,0,dcfdbc92f23847d6e243e23095883ba99819f6709ce5e579b34abfab2787135a
139,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,fa24b9526e85b92f0206b80f2844879704d21f2922789776da05156a43edf577,1,3267446,NaN,NaN,NaN,0,423e5dacd368e19d46eb1b6a8d0e55c341f57bd4a7b1d79d28f6305c80bda581


In [79]:
df2[df2['TxHash'] == TX2_IN]

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
44,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,4dd84cf29e802861f44b64d9cfe503b49e008add8ad3923edf395d4d0459eba1,1,3000000,NaN,NaN,NaN,0,7599387d26530d1392769a7c1cd04ce2f27de038c21818a17d4880d94e6c2976


In [80]:
# check wallet1, wallet2 and contract, redeem address
df = pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,4dd84cf29e802861f44b64d9cfe503b49e008add8ad3923edf395d4d0459eba1,0,243417177,NaN,NaN,None,0,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN
2,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,0,248856931,NaN,NaN,None,0,NaN
3,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747932,Party2,1,NaN


In [81]:
TX2_CONTRACT     = TX2_IN + "#1"
TX2_PARTY1_ADA   = df.iloc[0]['TxHash'] + '#0'
TX2_PARTY1_TOKEN = df.iloc[1]['TxHash'] + '#1'
TX2_PARTY2_ADA   = df.iloc[2]['TxHash'] + '#0'
TX2_PARTY2_TOKEN = df.iloc[3]['TxHash'] + '#1'
print(TX2_CONTRACT)
print(TX2_PARTY1_ADA)
print(TX2_PARTY1_TOKEN)
print(TX2_PARTY2_ADA)
print(TX2_PARTY2_TOKEN)

4dd84cf29e802861f44b64d9cfe503b49e008add8ad3923edf395d4d0459eba1#1
4dd84cf29e802861f44b64d9cfe503b49e008add8ad3923edf395d4d0459eba1#0
d007adb1b4ed647781eab93bb181f70195db93ae65dca353a5fc5a2985c65e6c#1
b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943#0
b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943#1


In [ ]:
# TX2_CONTRACT     = TX2_IN + "#1"
# TX2_PARTY1_ADA   = TX2_IN + "#0"
# TX2_PARTY1_TOKEN = TX1_IN + "#1"
# TX2_PARTY2_ADA   = "09c3f748b27145d998b392058269cb4f8716f4ec6c5f97deeef6ad5ae409d44b" + "#3"
# TX2_PARTY2_TOKEN = TX1_IN + "#2"

In [82]:
cmd = [marloweCLI, "run", "execute",         # Run a Marlowe transaction
       "--testnet-magic", MAGIC,             # '1567'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--marlowe-in-file", "tx-1.marlowe",  # 'tx-1.marlowe'
       "--tx-in-marlowe", TX2_CONTRACT,      # 'f773c71f9b58dd95fd470aedd9df800838e841aef807102abcd87e3c881fd373#1'
       "--tx-in-collateral", TX2_PARTY1_ADA, # 'f773c71f9b58dd95fd470aedd9df800838e841aef807102abcd87e3c881fd373#0'
       "--tx-in", TX2_PARTY1_ADA,            # 'f773c71f9b58dd95fd470aedd9df800838e841aef807102abcd87e3c881fd373#0'
       "--tx-in", TX2_PARTY1_TOKEN,          # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9#1'
       "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--required-signer", PARTY1_SKEYFILE, # 'wallet1.skey'
       "--marlowe-out-file", "tx-2.marlowe", # 'tx-2.marlowe'
       "--tx-out", PARTY1_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY1_TOKENASSETNAME,
       # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party1'
       "--out-file", "tx-2.raw",             # 'tx-2.raw'
       "--print-stats",
       "--submit", "600"]                    # '600'
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81"


Fee: Lovelace 1207675
Size: 13578 / 16384 = 82%
Execution units:
  Memory: 5666244 / 14000000 = 40%
  Steps: 1497742887 / 10000000000 = 14%



In [83]:
TX3_IN = result.stdout.split('"')[1]
TX3_IN

'110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81'

In [84]:
# check wallet1, wallet2 and contract, redeem address
df3 = pd.concat([query_df(CONTRACT_ADDRESS)]).reset_index(drop=True)
df3[df3['TxHash'] == TX3_IN]

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
6,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81,1,203000000,NaN,NaN,NaN,0,e62bd39502f50049286f72c99a1a6185ffdb764ba1d7c506a95309734f931a1c


In [86]:
df = pd.concat([query_df(PARTY1_ADDRESS)]).reset_index(drop=True)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81,0,41209502,NaN,NaN,None,0,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81,2,3000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN


In [88]:
TX3_CONTRACT     = TX3_IN + "#1"
TX3_PARTY1_ADA   = df.iloc[0]['TxHash'] + '#0'
TX3_PARTY1_TOKEN = df.iloc[1]['TxHash'] + '#2'
print(TX3_CONTRACT)
print(TX3_PARTY1_ADA)
print(TX3_PARTY1_TOKEN)

110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81#1
110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81#0
110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81#2


In [ ]:
# check wallet1, wallet2 and contract, redeem address
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

In [ ]:
TX3_CONTRACT     = TX3_IN + "#1"
TX3_PARTY1_ADA   = TX3_IN + "#0"
TX3_PARTY1_TOKEN = TX3_IN + "#2"

### prepare contract (stage2-3)

In [89]:
cmd = [marloweCLI, "run", "prepare",                       # Prepare the next step
       "--marlowe-file", "tx-2.marlowe",                   # 'tx-2.marlowe'
       "--deposit-account", PARTY2_TOKENASSETNAME, # 'Party2'
       "--deposit-party", PARTY2_TOKENASSETNAME,   # 'Party2'
       "--deposit-amount", str(PARTY2_AMOUNT),             # '100000000'
       "--invalid-before", str(NOW),                       # '1658239769000'
       "--invalid-hereafter", str(NOW+9*HOUR),             # '1658272169000'
       "--out-file", "tx-3.marlowe",                       # 'tx-3.marlowe'
       "--print-stats"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1672929783000},POSIXTime {getPOSIXTime = 1672962183999}), txInputs = [NormalInput (IDeposit "Party2" "Party2" (Token "" "") 100000000)]}
Payment 1
  Acccount: "Party1"
  Payee: Party "Party2"
  Ada: Lovelace {getLovelace = 200000000}
Payment 2
  Acccount: "Party2"
  Payee: Party "Party1"
  Ada: Lovelace {getLovelace = 100000000}
Payment 3
  Acccount: "Party1"
  Payee: Party "Party1"
  Ada: Lovelace {getLovelace = 3000000}

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1672929783000},POSIXTime {getPOSIXTime = 1672962183999})

Datum size: 59



In [ ]:
# cmd = [marloweCLI, "run", "prepare",                       # Prepare the next step
#        "--marlowe-file", "tx-2.marlowe",                   # 'tx-2.marlowe'
#        "--deposit-account", "Role="+PARTY2_TOKENASSETNAME, # 'Party2'
#        "--deposit-party", "Role="+PARTY2_TOKENASSETNAME,   # 'Party2'
#        "--deposit-amount", str(PARTY2_AMOUNT),             # '100000000'
#        "--invalid-before", str(NOW),                       # '1658239769000'
#        "--invalid-hereafter", str(NOW+9*HOUR),             # '1658272169000'
#        "--out-file", "tx-3.marlowe",                       # 'tx-3.marlowe'
#        "--print-stats"]
# result = subprocess.run(cmd, capture_output=True, text=True)
# print(result.stdout)
# print(result.stderr)

### deposit 100 ADA (step4)

In [90]:
# check wallet1, wallet2 and contract, redeem address
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash,210375bd6fd5d03cde2707b2e87114837d45f7ca7ee80278f945f38e,476c6f6265
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81,0,41209502,NaN,NaN,None,0,NaN,NaN,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81,2,3000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN,NaN,NaN
2,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,0,248856931,NaN,NaN,None,0,NaN,NaN,NaN
3,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,b1ae82baa5a421a060e997e6ff7a4070c521d39fbe962cf04730ec226d589943,1,2000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747932,Party2,1,NaN,NaN,NaN
4,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,00cbd222c84bbff42b1c49f718ae21bda1b7ce0488354f51bc6ba815bfa9d2a4,1,1500000,NaN,NaN,NaN,0,3e1b58fb13d4abbf5ee28bc3b911fe015c4fd17dbdcaf0e9eb8da328c7058961,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
142,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,f79c8195c449d6db92e9c3bb517918bf261e69208c9daee27f0ac3cd5d86b078,1,2638045,NaN,NaN,NaN,0,dcfdbc92f23847d6e243e23095883ba99819f6709ce5e579b34abfab2787135a,NaN,NaN
143,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,fa24b9526e85b92f0206b80f2844879704d21f2922789776da05156a43edf577,1,3267446,NaN,NaN,NaN,0,423e5dacd368e19d46eb1b6a8d0e55c341f57bd4a7b1d79d28f6305c80bda581,NaN,NaN
144,addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv,fce71f23bbf3e51ff7189558a27aca64ef99b16a3b5087ae02b6d1f16cc8589b,1,1500000,NaN,NaN,NaN,0,53e67e13b129d29d8e39a2e4cdf7411de0ebe25521c04dec561a3d67daefd743,NaN,NaN
145,addr_test1wpyswm4tyqjrmj2xy5glhx9fe7m3n7rwj6fz3qfekly3muckehcu9,3ea83fd7d9ecf8b99763de66e00a9dd47e06d5fbeff308bbba4f9fa8d0ebb264,2,1193870,NaN,NaN,NaN,0,NaN,{'5377616e': 500},NaN


In [91]:
# Run a marlowe contract on the blockchain 'run execute'
cmd = [marloweCLI, "run", "execute", 
       "--testnet-magic", MAGIC,
       "--socket-path", SOCKET,
       "--marlowe-in-file", "tx-2.marlowe",
       "--tx-in-marlowe", TX3_CONTRACT,
       "--tx-in-collateral", TX2_PARTY2_ADA,
       "--tx-in", TX2_PARTY2_ADA,
       "--tx-in", TX2_PARTY2_TOKEN,
       "--change-address", PARTY2_ADDRESS,
       "--required-signer", PARTY2_SKEYFILE,
       "--marlowe-out-file", "tx-3.marlowe",
       "--tx-out", PARTY2_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY2_TOKENASSETNAME,
       "--out-file", "tx-3.raw",
       "--print-stats",
       "--submit", "600"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "f5e47b068ebe81fa5e86b24710d7122afded911574a1b9585351d6afcdace61a"


Fee: Lovelace 1302325
Size: 13445 / 16384 = 82%
Execution units:
  Memory: 7006606 / 14000000 = 50%
  Steps: 1819010451 / 10000000000 = 18%



In [92]:
TX4_IN = result.stdout.split('"')[1]
TX4_IN

'f5e47b068ebe81fa5e86b24710d7122afded911574a1b9585351d6afcdace61a'

In [ ]:
# # check wallet1, wallet2 and contract, redeem address
# pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

In [93]:
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81,0,41209502,NaN,NaN,None,0,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81,2,3000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN
2,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,f5e47b068ebe81fa5e86b24710d7122afded911574a1b9585351d6afcdace61a,0,146554606,NaN,NaN,None,0,NaN
3,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,f5e47b068ebe81fa5e86b24710d7122afded911574a1b9585351d6afcdace61a,3,3000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747932,Party2,1,NaN


In [96]:
TX_PARTY1_ADA   = TX3_IN + "#0"
TX_PARTY1_TOKEN = TX3_IN + "#2"
TX_PARTY2_ADA   = TX4_IN + "#0"
TX_PARTY2_TOKEN = TX4_IN + "#3"
print(TX_PARTY1_ADA)
print(TX_PARTY1_TOKEN)
print(TX_PARTY2_ADA)
print(TX_PARTY2_TOKEN)

110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81#0
110de87846a0b696ac61fe866aeaa8f6e3ce9f8c6135316c02c789d5dfb3ec81#2
f5e47b068ebe81fa5e86b24710d7122afded911574a1b9585351d6afcdace61a#0
f5e47b068ebe81fa5e86b24710d7122afded911574a1b9585351d6afcdace61a#3


### withdraw (step5)

In [ ]:
# tx-in-collateral is the wallet with funds from the PARTY_1_ROLE just some ADA for fee available to bre used for allowing a transaction to occur
# tx-in is the original wallet with only ADA currency that needs to be consumed and replaced after a Transaction and is to allow for new funds to appear and change this storage of ADA
# tx-in (second) is to allow for using the token for that is needed as defined in the contract to give permission using the token to do a transaction...

In [97]:
cmd = [marloweCLI, "run", "withdraw", 
       "--testnet-magic", MAGIC,
       "--socket-path", SOCKET,
       "--marlowe-file", "tx-3.marlowe",      # last contract version
       "--role-name", PARTY1_TOKENASSETNAME,
       "--tx-in-collateral", TX_PARTY1_ADA,
       "--tx-in", TX_PARTY1_ADA,
       "--tx-in", TX_PARTY1_TOKEN,
       "--change-address", PARTY1_ADDRESS,
       "--required-signer", PARTY1_SKEYFILE,
       "--tx-out", PARTY1_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY1_TOKENASSETNAME,   # needed against dos attack of 1000 calls
       "--out-file", "/dev/null",
       "--print-stats",
       "--submit", str(600)]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "944d783d88e5b5ba9042d3bef23f9e3f632e6690620b2264408f4a0d187404fb"


Fee: Lovelace 455329
Size: 3185 / 16384 = 19%
Execution units:
  Memory: 1799270 / 14000000 = 12%
  Steps: 500109567 / 10000000000 = 5%



In [98]:
cmd = [marloweCLI, "run", "withdraw", 
       "--testnet-magic", MAGIC,
       "--socket-path", SOCKET,
       "--marlowe-file", "tx-3.marlowe",      # last contract version
       "--role-name", PARTY2_TOKENASSETNAME,
       "--tx-in-collateral", TX_PARTY2_ADA,
       "--tx-in", TX_PARTY2_ADA,
       "--tx-in", TX_PARTY2_TOKEN,
       "--change-address", PARTY2_ADDRESS,
       "--required-signer", PARTY2_SKEYFILE,
       "--tx-out", PARTY2_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY2_TOKENASSETNAME,   # needed against dos attack of 1000 calls
       "--out-file", "/dev/null",
       "--print-stats",
       "--submit", str(600)]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "e6cb971f8dfe12676d7b610708c3e86207112817d8d2592fe8486b089d520aad"


Fee: Lovelace 459391
Size: 3185 / 16384 = 19%
Execution units:
  Memory: 1854070 / 14000000 = 13%
  Steps: 512594488 / 10000000000 = 5%



In [99]:
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,944d783d88e5b5ba9042d3bef23f9e3f632e6690620b2264408f4a0d187404fb,0,40754173,NaN,NaN,None,0,NaN
1,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,944d783d88e5b5ba9042d3bef23f9e3f632e6690620b2264408f4a0d187404fb,1,103000000,NaN,NaN,None,0,NaN
2,addr_test1vpqflumj2n8d43k22k4v9xfzvpd70jwfra7cruv04uht0tsj0t9x5,944d783d88e5b5ba9042d3bef23f9e3f632e6690620b2264408f4a0d187404fb,2,3000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747931,Party1,1,NaN
3,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,e6cb971f8dfe12676d7b610708c3e86207112817d8d2592fe8486b089d520aad,0,146095215,NaN,NaN,None,0,NaN
4,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,e6cb971f8dfe12676d7b610708c3e86207112817d8d2592fe8486b089d520aad,1,200000000,NaN,NaN,None,0,NaN
5,addr_test1vqyq7qum3fwuyc6pge0a8td0enx5gctnh43h2zntmkc994qz04pwj,e6cb971f8dfe12676d7b610708c3e86207112817d8d2592fe8486b089d520aad,2,3000000,8050cb8a6ee792c13aebc35d72e9b5c840ca2e4233d67d16de6b9ff8,506172747932,Party2,1,NaN


In [ ]:
# # check wallet1, wallet2 and contract, redeem address
# pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

In [ ]:
# so now 100 ADA has been withdrawn or redeemed to the Party1
# also the 3 ADA used for the script activation is returned to the initiator of the contract.

In [ ]:
https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli/examples/swap#transaction-2-first-party-deposits-tokens-into-the-contract

### Function for Pandas Dataframe

In [ ]:
cardanoCliDataframe(PARTY_1_ADDRESS, MAGIC, 'full')

In [ ]:
cardanoCliDataframe(PARTY_1_ADDRESS, MAGIC)

In [ ]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", PARTY1_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [ ]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", PARTY2_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [ ]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", CONTRACT_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [ ]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", REDEEM_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [ ]:
regex = re.compile(r'(\S{64})\s+(\d+)\s+(\d{6,})\s+(\S+)\s+[+]\s+(.+)')

In [ ]:
result.stdout

In [ ]:
df = pd.DataFrame(regex.findall(result.stdout), columns=['TxHash', 'TxIx', 'AmountLovelace', 'Currency', 'tail'])
df

In [ ]:
df.join(df['tail'].str.replace('.', ' ', regex=False).str.replace(' + ', ' ', regex=False).str.replace('"', '', regex=False).str.split(' ', regex=False, expand=True), how='left')

In [ ]:
df['tail'].iloc[2] = 'TxOutDatumHash ScriptDataInAlonzoEra "e13945ede9a5b560e25c175068ce0b955402ec7558090fbf510d7470d840e89f"'

In [ ]:
df

In [ ]:
df.join(df['tail'].str.replace('.', ' ', regex=False).str.replace(' + ', ' ', regex=False).str.replace('"', '', regex=False).str.split(' ', regex=False, expand=True), how='left')

In [ ]:
df = df.join(df['tail'].str.replace('.', ' ', regex=False).str.replace(' + ', ' ', regex=False).str.replace('"', '', regex=False).str.split(' ', regex=False, expand=True), how='left')
df

In [ ]:
df.columns = ['TxHash', 'TxIx', 'AmountLovelace', 'Currency', 'tail', 'TokenAmount', 'TokenPolicyId', 'TokenAssetNameHex', 'TxOut']
df

In [ ]:
df = df.drop(['tail', 'TxOut'], axis=1)
df

In [ ]:
df = df.replace({'TokenAmount':{'TxOutDatumNone': 0}})

In [ ]:
def hex_to_ascii(s):
    try:
        return bytearray.fromhex(s).decode()
    except ValueError:
        return None

In [ ]:
df['TokenAssetNameAscii'] = df['TokenAssetNameHex'].astype('str').apply(hex_to_ascii)

In [ ]:
df['TxHashSmall'] = df['TxHash'].str[-4::1] # get last 4 characters

In [ ]:
df['TokenPolicyIdSmall'] = df['TokenPolicyId'].str[-4::1] # get last 4 characters

In [ ]:
df['AmountAda'] = df['AmountLovelace'].astype(float)/1000000

In [ ]:
df[['TxHashSmall', 'TxIx', 'AmountAda', 'TokenAmount', 'TokenPolicyIdSmall']]

In [ ]:
df

In [ ]:
'TokenPolicyId', 'TokenAssetNameHex', 'TokenAssetNameAscii'

In [513]:
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)])

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN
0,addr_test1wqzctw6fur749cqp8ae9y8e6l6g9uvs9t4pv7thp0zy38cssc9g3z,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,1,3000000,NaN,NaN,NaN,0,0e810f91611be5f0023233e5aa189dd099ad96d05437ffe0bc59a0f9a0137d64
0,addr_test1wqzuqf8hvl9d47d6kftykktrgszgu3yznrrpdgha9naxcmgr8le36,NaN,0,0,NaN,NaN,NaN,0,NaN


In [489]:
query_df(PARTY1_ADDRESS)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,NaN,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,1.0,NaN


In [490]:
query_df(PARTY2_ADDRESS)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount,datumhash
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,1.0,NaN
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,NaN,NaN


In [491]:
query_df(CONTRACT_ADDRESS)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount,datumhash
0,addr_test1wqzctw6fur749cqp8ae9y8e6l6g9uvs9t4pv7thp0zy38cssc9g3z,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,1,3000000,NaN,NaN,NaN,0e810f91611be5f0023233e5aa189dd099ad96d05437ffe0bc59a0f9a0137d64


In [492]:
query_df(REDEEM_ADDRESS)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount,datumhash
0,addr_test1wqzuqf8hvl9d47d6kftykktrgszgu3yznrrpdgha9naxcmgr8le36,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# interesting resource
# https://github.com/input-output-hk/marlowe-cardano/blob/c91f711b5132620dae7713dcc5a8607605b38f8f/marlowe-cli/cookbook/english-auction.ipynb

In [476]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", REDEEM_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------



In [477]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", REDEEM_ADDRESS, "--out-file", 'party1_address.json']
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [478]:
!cat party1_address.json

{}

In [479]:
df = pd.read_json('party1_address.json', orient='index').reset_index()
df

,index


In [469]:
# replace feature names with interpretable naming (others are left as-is) 
df = df.rename(columns={'index': 'TxHash', 'datumhash': 'datumhash', 'address': 'address', 'value': 'value'})
df

,TxHash


In [470]:
# df.columns = ['TxHash', 'address', 'value']
# df

In [471]:
df = df.join(df['TxHash'].str.split('#', regex=False, expand=True), how='left')
df

AttributeError: Can only use .str accessor with string values!

In [445]:
df = df.drop(['TxHash'], axis=1)
df

,address,value,0,1
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,{'6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508': {'506172747932'...,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,{'lovelace': 500000000},ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3


In [446]:
# replace feature names with interpretable naming (others are left as-is) 
df = df.rename(columns={0: 'TxHash', 1: 'TxIx'})
df

,address,value,TxHash,TxIx
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,{'6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508': {'506172747932'...,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,{'lovelace': 500000000},ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3


In [447]:
# df.columns = ['address', 'value', 'TxHash', 'TxIx']
# df

In [448]:
df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000.0
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000.0


In [449]:
df['lovelace'] = df['lovelace'].astype('int')
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000


In [450]:
df['TxIx'] = df['TxIx'].astype('int')
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000


In [451]:
# drop all columns with all values NA
df = df.dropna(axis=1, how='all')
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000


In [452]:
# if the last column name word size is 56 characters wide:
# cn = df.iloc[:, -1].name
try:
    cn = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace'])]].iloc[:, -1].name
    if len(cn) == 56:
        print("yes, len(cn) == 56")
        # make a copy
        df['TokenPolicyId'] = df[cn]
        # write the column name only on True location
        df['TokenPolicyId'] = df['TokenPolicyId'].notna().replace({True: cn, False: np.nan})
except:
    print("except happened")
    df['TokenPolicyId'] = np.nan
    df['TokenAssetNameHex'] = np.nan
    df['TokenAmount'] = np.nan
    pass

yes, len(cn) == 56


In [453]:
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace,TokenPolicyId
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000,NaN


In [454]:
df = pd.concat([df.drop([cn], axis=1), df[cn].apply(pd.Series)], axis=1)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,506172747932,0
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,1.0,NaN
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,NaN


In [455]:
# drop all columns with all values NA
df = df.dropna(axis=1, how='all')
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,506172747932
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,1.0
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN


In [456]:
# cn = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId'])]].iloc[:, -1].name
# # make a copy 
# df['TokenAssetNameHex'] = df[cn]
# # write the column name only on True location
# df['TokenAssetNameHex'] = df['TokenAssetNameHex'].notna().replace({True: cn, False: np.nan})

In [457]:
two = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId'])]]
two

,506172747932
0,1.0
1,NaN


In [458]:
tmp = two.stack().apply(pd.Series).reset_index()
tmp

,level_0,level_1,0
0,0,506172747932,1.0


In [459]:
tmp = tmp.set_index('level_0')
tmp

,level_1,0
level_0,,
0,506172747932,1.0


In [460]:
tmp.columns = ['TokenAssetNameHex', 'TokenAmount']
tmp

,TokenAssetNameHex,TokenAmount
level_0,,
0,506172747932,1.0


In [461]:
df = pd.concat([df, tmp], axis=1)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,506172747932,TokenAssetNameHex,TokenAmount
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,1.0,506172747932,1.0
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,NaN,NaN


In [462]:
# df['TokenAmount'] = df['TokenAmount'].astype('int')
# df

In [463]:
df = df.drop(tmp['TokenAssetNameHex'], axis=1)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,1.0
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,NaN


In [ ]:
# df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId'])]].stack()
# # make a copy 
# df['TokenAssetNameHex'] = df[cn]
# # write the column name only on True location
# df['TokenAssetNameHex'] = df['TokenAssetNameHex'].notna().replace({True: cn, False: np.nan})

In [85]:
# df

,address,TxHash,TxIx,lovelace,TokenPolicyId,506172747932,506172747931,TokenAssetNameHex
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,e595b9fc55e0df7f8e0b8522628b8150e37cf249b4265e651e089cbf150ef3ea,0,479823763,NaN,NaN,NaN,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,e595b9fc55e0df7f8e0b8522628b8150e37cf249b4265e651e089cbf150ef3ea,2,10000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,1.0,NaN,NaN
2,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,e595b9fc55e0df7f8e0b8522628b8150e37cf249b4265e651e089cbf150ef3ea,1,10000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,NaN,1.0,506172747931


In [ ]:
# df = df.rename(columns={cn: 'TokenAmount'})
# df

In [ ]:
clean up and send your remaining Preview TestADA back to the faucet address for other people to use
add code for this below...

In [ ]:
https://developers.cardano.org/docs/get-started/running-cardano/

cardano-node run \
   --topology topology.json \
   --database-path ~/cardano/db/ \
   --socket-path /tmp/node.socket \
   --host-addr 0.0.0.0 \
   --config config.json

export PATH=$PATH:~/cardano
export CARDANO_NODE_SOCKET_PATH="/tmp/node.socket"

<wait a bit, if below fails>
cardano-cli query tip --testnet-magic 2

cardano-wallet serve \
   --listen-address 0.0.0.0 \
   --port 8090 \
   --node-socket /tmp/node.socket \
   --testnet byron-genesis.json \
   --database ~/cardano/wallet-db \
   --token-metadata-server https://metadata.cardano-testnet.iohkdev.io
   
   
curl http://localhost:8090/v2/network/information
curl http://20.123.241.153:8090/v2/network/information
http://20.123.241.153:8090/v2/network/information

get testADA: https://docs.cardano.org/cardano-testnet/tools/faucet

Run a Docker container with cardano-node and cardano-wallet services:  

```sh
cd ~/cardano
wget https://github.com/input-output-hk/cardano-wallet/raw/master/docker-compose.yml

# download, install and run the cardano-wallet backend node using docker (autorestart enabled)
NETWORK=preprod docker-compose up -d

# init sync blockchain, CPU intensive, +7GB mem, +30GB, wait some hours, sync_progress
curl http://localhost:8090/v2/network/information
curl http://20.86.118.190:8090/v2/network/information # (optional: open port 8090)

# if you want to stop all cardano backend services (preserve data)
NETWORK=preprod docker-compose down
```